Before going into the Kernel,

I REFERED ON THIS KERNEL https://www.kaggle.com/willkoehrsen/a-complete-introduction-and-walkthrough
I translated into Korean for my studying

I'D LIKE TO GIVE HUGE THANKS TO HIM

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv("../input/train.csv") 
test = pd.read_csv("../input/test.csv")

# Costa Rican Household Poverty Level Prediction<br />
Welcome to another Kaggle challenge! The objective of the Costa Rican Household Poverty Level Prediction contest is to develop a machine learning model that can predict the poverty level of households using both individual and household characteristics. This "data science for good" project offers the opportunity to put our skills towards a task more beneficial to society than getting people to click on ads!<br />
=> 가구 개개인 과 가구의 특징을 이용하여 가구의 빈층단계를 예측하는 모델을 만드는 것

* In this notebook, we will walk through a complete machine learning solution: <br />
* first, get introduced to the problem, [문제를 도출]<br />
* then perform a thorough Exploratory Data Analysis of the dataset, [EDA를 통한 데이터 탐색] <br />
* work on feature engineering, <br />
* try out multiple machine learning models, [여러개의 머신러닝 모델에 대입]<br />
* select a model, [모델을 선택] <br />
* work to optimize the model, [모델을 최적화] <br /> 
* and finally, inspect the outputs of the model and draw conclusions. [결론도출] <br />

__While this notebook may not get us to the top of the leaderboard, it is meant to be used as a teaching tool to give you a solid foundation to build on for future machine learning projects. Kaggle projects can teach us a lot about machine learning, but several of the strategies used to get to the very top of the leaderboard are not best practices, so here we'll stick to building a very good - although not quite first place - machine learning solution. While Kaggle projects are competitions, I think they are best described as "a machine learning education" disguised as a contest!"

If you are looking to follow-up on this work, I have additional work including a kernel on using Automated Feature Engineering with Featuretools for this problem (with slightly higher leaderboard score). (If you enjoy my writing style and explanations, I write for Towards Data Science)

Problem and Data Explanation <br />
The data for this competition is provided in two files: train.csv and test.csv. <br />
The training set has 9557 rows and 143 columns [훈련세트의 정보] <br />
while the testing set has 23856 rows and 142 columns. [테스트셋의 정보] <br />
Each row represents one individual and each column is a feature, either unique to the individual[개인의 고유값], or for the household of the individual.[각 가구별 고윳값] <br />
The training set has one additional column,  Target, which represents the poverty level on a 1-4 scale and is the label for the competition. <br />
[우리가 예측하고자 하는 결과 값이 들어있는 컬럼] <br />
A value of 1 is the most extreme poverty. <br />

This is a supervised multi-class classification machine learning problem: <br />
=> 지도학습 && 다중분류 <br />

Supervised: provided with the labels for the training data <br />
Multi-class classification: Labels are discrete values with 4 classes <br />

Objective <br />
The objective is to predict poverty on a household level. <br /> 
=> 목표는 가구단위로 빈곤단계를 구분하는 것 <br />
We are given data on the individual level with each individual having unique features but also information about their household. <br />
=> 정보는 가구원 개개인에 대한 유니크한 정보를 포함하며 가구 전체의 유니크한 정보도 포함한다 <br />
In order to create a dataset for the task, we'll have to perform some aggregations of the individual data for each household. <br /> 
=> 그래서 위를 위한 데이터셋을 만들기 위해서는 각각의 가정을 개개인의 데이터로 합쳐둘 필요가 있다 <br />
Moreover, we have to make a prediction for every individual in the test set,  <br />
=> 테스트셋에 있는 모든 개별 인원들에게 예측을 실행해야 한다 <br />
but "ONLY the heads of household are used in scoring" which means we want to predict poverty on a household basis <br />.
=> 하지만 무조건 가장에 대해서만 점수를 반영한다 <br />

Important note: <br />
while all members of a household should have the same label in the training data, <br /> 
=> 가구의 모든 구성원들이 무조건 같은 단계를 받아야 한다 <br />
there are errors where individuals in the same household have different labels. <br /> 
=> 하지만 일부 가구의 일부 구성원들은 다른 구성원과는 다른 단계를 가지고 있는 경우도 있다. <br />
In these cases, we are told to use the label for the head of each household, <br />
=> 이경우에는 우리는 가구의 가장의 label을 이용할 것이다 <br />
which can be identified by the rows where parentesco1 == 1.0. <br />

We will cover how to correct this in the notebook (for more info take a look at the competition main discussion). <br />
=> 왜 이 접근법이 맞는지는 Discussion에서 참고하길 바란다 <br />

The Target values represent poverty levels as follows: <br />
=> 단계는 아래와 같다 <br />

1 = extreme poverty [극빈층] <br />
2 = moderate poverty [적정 수준의 빈곤]<br />
3 = vulnerable households [취약계층] <br />
4 = non vulnerable households [취약계층이 아님]<br />

The explanations for all 143 columns can be found in the competition documentation, but a few to note are below: <br />
=> 143개의 columns를 가지고 있다 그 중 일부 컬럼에 대하 설명 <br />

Id: a unique identifier for each individual, this should not be a feature that we use! <br />
=> 개인에게 주어지는 식별자 <br />
idhogar: a unique identifier for each household. This variable is not a feature, but will be used to group individuals by household as all individuals in a household will have the same identifier. <br />
=> 각각의 가구를 구분하는 식별자 <br />
parentesco1: indicates if this person is the head of the household. <br />
=> 가장을 나타내는 컬럼<br />
Target: the label, which should be equal for all members in a household<br />
=> 모든 가구들은 라벨이 모두 동일해야 한다.

When we make a model, we'll train on a household basis with the label for each household the poverty level of the head of household. <br />
=> 예측을 하는데 있어서 가장의 정보만 사용할 것임<br />
The raw data contains a mix of both household and individual characteristics and for the individual data, <br />
=> 기존의 데이터는 개개인에 대한 정보와 가구에 대한 정보를 함께가지고 있음<br />
we will have to find a way to aggregate this for each household. <br />
=> 각각의 가구별로 확인 가능하도록 데이터를 합치는 방법을 모색해야 한다<br />
Some of the individuals belong to a household with no head of household which means that unfortunately we can't use this data for training. <br />
=> 몇몇의 가구는 가장이 없는 경우가 있는데, 이 경우는 사용하지 않도록 한다<br />
These issues with the data are completely typical of real-world data and hence this problem is great preparation for the datasets you'll encounter in a data science job!<br />
=> 위의 문제들은 실제에서 가져오는 데이터에서 많이 일어날 수 있는 문제임.

Metric [평가지표]<br />
Ultimately we want to build a machine learning model that can predict the integer poverty level of a household. <br />
궁극적으로 우리가 원하는 것은 각각의 가구의 빈곤레벨을 예측하는 것<br />
Our predictions will be assessed by the Macro F1 Score. You may be familiar with the standard F1 score for binary classification problems which is the harmonic mean of precision and recall:<br />
우리는 Macro F1 Score를 사용할 것이다. 기존의 F1 score는 정밀도와 재현율의 조화평균한 값이다

** 조화평균 [역수들을 산술평균(Average) 한 것들의 역수]<br />
https://blog.naver.com/hyuiin/221367087739 

F1=21recall+1precision=2⋅precision⋅recallprecision+recall
 
For mutli-class problems, we have to average the F1 scores for each class. The macro F1 score averages the F1 score for each class without taking into account label imbalances.

Macro F1=F1 Class 1+F1 Class 2+F1 Class 3+F1 Class 44
 
In other words, the number of occurrences of each label does not figure into the calculation when using macro (while it does when using the "weighted" score). <br />
=> 각 라벨이 등장한 횟수의 경우에는 계산에 적용되지 않을 것이다 macro를 사용하게 되면<br />
(For more information on the differences, look at the Scikit-Learn Documention for F1 Score or this Stack Exchange question and answers. If we want to assess our performance, we can use the code:

from sklearn.metrics import f1_score<br />
f1_score(y_true, y_predicted, average = 'macro`)

For this problem, the labels are imbalanced, which makes it a little strange to use macro averaging for the evaluation metric, <br />
=> 우리가 예측하려는 라벨이 불균형이 있음 (평가 지표로 macro averaging을 사용하는 것이 조금 이상하게 느껴질 수 있음)<br />
but that's a decision made by the organizers and not something we can change! In your own work, you want to be aware of label imbalances and choose a metric accordingly.<br />
=> 그러나 그것은 이미 지정된 것이기 때문에 상관할 필요 없음 단지 라벨의 분균형과 그것에 해당하는 평가지표 방법을 고른 것이라고 생각하면 된다.

Roadmap<br />
The end objective is a machine learning model that can predict the poverty level of a household. <br />
=> 우리의 최종목표는 빈곤게층을 예측하는 것이다<br />
However, before we get carried away with modeling, it's important to understand the problem and data. <br />
=> 알아보기전에 우리가 직면한 문제와 데이터에 대해서 알아보도록 하자<br />
Also, we want to evaluate numerous models before choosing one as the "best" and after building a model, we want to investigate the predictions. Our roadmap is therefore as follows:<br />
=> 우리의 로드맵은 아래의 순서와 같다

Understand the problem (we're almost there already)<br />
Exploratory Data Analysis<br />
Feature engineering to create a dataset for machine learning<br />
Compare several baseline machine learning models<br />
Try more complex machine learning models<br />
Optimize the selected model<br />
Investigate model predictions in context of problem<br />
Draw conclusions and lay out next steps

The steps laid out above are iterative meaning that while we will go through them one at a time, we might go back to an earlier step and revisit some of our decisions. <br />
In general, data science is a non-linear pracice where we are constantly evaluating our past decisions and making improvements. <br />
In particular, feature engineering, modeling, and optimization are steps that we often repeat because we never know if we got them right the first time!

Getting Started<br />
We have a pretty good grasp of the problem, so we'll move into the Exploratory Data Analysis (EDA) and feature engineering. <br />
For the EDA we'll examine any interesting anomalies, trends, correlations, or patterns that can be used for feature engineering and for modeling. <br />
=> 우리는 특성공학과 모델릴에 쓰이는 흥미로운 잘못된 점, 추세, 상관관계 도는 패턴을 탐구할 것입니다<br />
We'll make sure to investigate our data both quantitatively (with statistics) and visually (with figures).<br />
=> 우리의 데이터를 양적으로 그리고 시각적으로 탐사해봅시다.

Once we have a good grasp of the data and any potentially useful relationships, <br />
=> 위의 탐사를 성공적으로 끝낸 후에야<br />
we can do some feature engineering (the most important part of the machine learning pipeline) and establish a baseline model. <br />
=> 특성공학이나 기반모델을 설게할 수 있을 것입니다<br />
This won't get us to the top of the leaderboard, but it will provide a strong foundation to build on!<br />

With all that info in mind (don't worry if you haven't got all the details), let's get started!

Imports
We'll use a familiar stack of data science libraries: Pandas, numpy, matplotlib, seaborn, and eventually sklearn for modeling.

In [ ]:
train.head()

That gives us a look at all of the columns which don't appear to be in any order. To get a quick overview of the data we use df.info().

In [ ]:
train.info()

This tells us there are 130 integer columns, 8 float (numeric) columns, and 5 object columns. <br />
=> 130개의 정수, 8개의 실수 그리고 5개의 객체 컬럼이 있습니다.<br />
The integer columns probably represent Boolean variables (that take on either 0 or 1) or ordinal variables with discrete ordered values. <br />
=> 정수인 컬럼은 불리언 값일 수도 있지만 순서를 가진 ordinal한 값일 가능성이 있습니다<br />
The object columns might pose an issue because they cannot be fed directly into a machine learning model.<br />
=> 객체컬럼은 머신러닝 모델에서 학습될 수 없기 때문에 어떠한 조치를 취해주어야 합니다

Let's glance at the test data which has many more rows (individuals) than the train. It does have one fewer column because there's no Target!<br />
=> 테스트 데이터셋을 보게 된다면 테스트 데이터셋에는 타겟 값이 없기 대문에 정수컬럼이 하나 없음을 알 수 있습니다.

In [ ]:
test.info()

Integer Columns<br />
Let's look at the distribution of unique values in the integer columns. For each column, we'll count the number of unique values and show the result in a bar plot.<br />
=> 위에서 말했듯이 정수컬럼의 유형을 구분하기 위해서 각각의 컬럼이 어떤 유니크한 개별값을 갖는지를 나타내보겠습니다.

In [ ]:
df_int_cols = train.select_dtypes('int').apply(lambda x:x.nunique()).reset_index()
df_int_cols.columns = ['var_name','count']
df_int_cols.groupby('count')['var_name'].size().plot(kind='bar')

plt.xlabel('Number of Unique Values'); plt.ylabel('Count');
plt.title('Count of Unique Values in Integer Columns');

The columns with only 2 unique values represent Booleans (0 or 1). <br />
=> 값이 두 개 밖에 없다는 것은 논리형을 뜻합니다.<br />
In a lot of cases, this boolean information is already on a household level.<br />
=> 이러한 논리형 정보는 가구 전체의 단계를 위해 이미 준비되었습니다<br />
For example, the refrig column says whether or not the household has a refrigerator. <br />
=> 그 중 하나로 refig는 가구가 냉장고를 가지는지 아닌지를 가르는 정보입니다.<br />
When it comes time to make features from the Boolean columns that are on the household level, we will not need to aggregate these. <br />
=> 이러한 논리형 정보는 가구당 정보이기 때문에 따로 취합할 필요성은 없는 듯 합니다<br />
However, the Boolean columns that are on the individual level will need to be aggregated.<br />
=> 그러나 일부 개개인을 위한 논리형 정보는 취합의 여지가 있습니다.

Float Columns [실수형 컬럼]<br />
Another column type is floats which represent continuous variables.<br />
=> 연속형 변수를 대표하는 실수형 컬럼입니다<br />
We can make a quick distribution plot to show the distribution of all float columns. <br />
=> 모든 정수형 컬럼의 분포를 간단하게 시각화해보려고 합니다<br />
We'll use an OrderedDict to map the poverty levels to colors because this keeps the keys and values in the same order as we specify (unlike a regular Python dictionary).<br />
=> 우리는 일반 딕셔너리 셩태와는 달리 정렬된 딕셔너리를 사용하려고 합니다. 왜냐하면 키와 값이 같은 같은 순서로 유지할 필요가 있기 때문입니다.

The following graphs shows the distributions of the float columns colored by the value of the Target. <br />
=> 다음의 그래프들은 타켓의 값 별로 칠해진 실수형 컬럼의 분포를 보여줄 것입니다.<br />
With these plots, we can see if there is a significant difference in the variable distribution depending on the household poverty level.<br />
=> 이 그래프들로 각각의 빈곤 레벨에 따른 개별 컬럼의 분포를 볼 수 있습니다. <br />

In [ ]:
from collections import OrderedDict

colors = OrderedDict({1: 'red', 2: 'orange', 3: 'blue', 4: 'green'})
poverty_mapping = OrderedDict({1: 'extreme', 2: 'moderate', 3: 'vulnerable', 4: 'non vulnerable'})

fig,ax = plt.subplots(4,2,figsize=[20,16])

for i,col in enumerate(train.select_dtypes('float').columns):
    ax = plt.subplot(4,2,i+1)
    for proverty_level,color in colors.items():
        sns.kdeplot(train.loc[train['Target']==proverty_level,col].dropna(),ax=ax,color=color,label=poverty_mapping[proverty_level])
    
    ax.set(title=f'{col.capitalize()} Distribution',xlabel=f'{col}',ylabel='Density')

Later on we'll calculate correlations between the variables and the Target to gauge the relationships between the features, <br />
=> 나중에 피쳐들 간에 관게를 보기 위해 각각의 변수들과 타켓의 상관관계를 계산할 것이다 <br />
but these plots can already give us a sense of which variables may be most "relevant" to a model. <br />
=> 이러한 그림은 이미 연관성에 대한 관점을 조금은 보여주는 듯하다<br />
For example, the meaneduc, representing the average education of the adults in the household appears to be related to the poverty level: a higher average adult education leads to higher values of the target which are less severe levels of poverty. <br />
=> 간단히 예를들면 meaneduc의 경우에는 빈곤계층 단계와 관련이 있어보이는 것 같다: 교육 수준 정도가 높을 수록 빈곤계층 단계가 더 4로 높아지는 것을 보여준다. 이는 교육수준이 높을 수록 빈곤의 수준이 낮게 됨을 볼 수 있다.<br />
The theme of the importance of education is one we will come back to again and again in this notebook!<br />
=> 교육의 중요성을 실감할 수 있다.

Object Columns [객체형 컬럼]<br />
The last column type is object which we can view as follows.

In [ ]:
train.select_dtypes('object').head()

The Id and idhogar object types make sense because these are identifying variables. <br />
Id와 idhogar가 객체형 자료인 것은 맞는 말인 것 같다<br />
However, the other columns seem to be a mix of strings and numbers which we'll need to address before doing any machine learning. According to the documentation for these columns:<br />
그러나 다른 컬럼들은 머신러닝 모델에 넣기전에 처리를 거쳐야 할 문자형과 숫자형 자료의 혼합형인 듯하다. 다음은 해당 컬럼들에 대한 정보이다<br />
dependency: Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)

edjefe: years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

edjefa: years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

These explanations clear up the issue. For these three variables, "yes" = 1 and "no" = 0. We can correct the variables using a mapping and convert to floats.<br />
위들 컬럼을 보면 yes는 1, no는 0 으로 mapping할 필요가 있어 보인다.

In [ ]:
mapping = {'yes':1,'no':0}

needToChange = train.select_dtypes('object').columns[-1:1:-1]

for df in [train,test]:
    for col in needToChange:
        df[col] = df[col].replace(mapping).astype(np.float64)
        
train.loc[:,needToChange].describe()

In [ ]:
fig,ax = plt.subplots(3,1,figsize=[16,18])

for i,col in enumerate(needToChange):
    ax = plt.subplot(3,1,i+1)
    for proverty_level,color in colors.items():
        sns.kdeplot(train.loc[train['Target']==proverty_level,col].dropna(),ax=ax,color=color,label=poverty_mapping[proverty_level])
    
    ax.set(title=f'{col.capitalize()} Distribution',xlabel=f'{col}',ylabel='Density')

These variables are now correctly represented as numbers and can be fed into a machine learning model.<br />
=> 이제 이 변수들은 완전히 숫저화 되엇으며 머신러닝 모델에 넣을 준비가 되었다

To make operations like that above a little easier, we'll join together the training and testing dataframes. <br />
=> 위의 작업을 통해 우리는 훈련데이터와 테스트셋 데이터에 모두 적용할 수 있었다<br />
This is important once we start feature engineering because we want to apply the same operations to both dataframes so we end up with the same features. <br />
=> 이는 feature engineering하는데 있어서 핵심적이다<br />
Later we can separate out the sets based on the Target.<br />
=> 이후에 타겟값을 기초로하여 훈련셋과 테스트셋을 다시 나눌 수 있을 것이다.

In [ ]:
# Add null Target column to test
test['Target'] = np.nan
data = train.append(test, ignore_index = True)

## Exploring Label Distribution<br />
Next, we can get an idea of how imbalanced the problem is by looking at the distribution of labels.<br />
=> 다음으로 어떻게 라벨 불균형 문제를 해결해야 하는지에 대한 아이디어를 얻을 수 있을 것이다.<br />
There are four possible integer levels, indicating four different levels of poverty. <br />
=> 알다시피 우리는 네개의 빈부수준을 나태내는 단계를 가지고 있다 <br />
To look at the correct labels, we'll subset only to the columns where parentesco1 == 1 because this is the head of household, the correct label for each household.<br />
=> 정확한 라벨들을 보기 위해서 우리는 가장의 서브셋만을 이용할 것이다

The bar plot below shows the distribution of training labels (since there are no testing labels).<br />
아래의 그림은 훈련 데이터셋의 타겟 라벨의 값의 분포를 보여준다

In [ ]:
heads = data.loc[data['parentesco1'] == 1].copy()

train_labels = data.loc[(data['Target'].notnull()) & (data['parentesco1'] == 1),['Target','idhogar']]

label_counts = train_labels['Target'].value_counts().sort_index()

label_counts.plot(kind='bar',figsize=[8,6],color=colors.values(),edgecolor='k',linewidth = 2)

plt.xlabel('Poverty Level'); plt.ylabel('Count'); 
plt.xticks([x - 1 for x in poverty_mapping.keys()], 
           list(poverty_mapping.values()), rotation = 60)
plt.title('Poverty Level Breakdown');

We are dealing with an imbalanced class problem (which makes it intriguing why the contest organizers choose the macro F1 score as the metric instead of weighted F1!). <br />
=> 위의 그림을 통해 우리는 클래스 불균형 문제를 다루고 있음을 알 수 있습니다 (이는 왜 컨테스트의 주최자가 가중 F1 점수가 아닌 Macro F1점수를 택했는지 의문점을 들게 합니다)<br />
There are many more households that classify as non vulnerable than in any other category. The extreme poverty class is the smallest (I guess this should make us optimistic!).<br />
=> 다른 카테고리보다 취약계층이 아닌 가구들이 더 많음을 알 수 있습니다. 그리고 극도의 빈곤게층은 가장 작음을 또한 할 수 있습니다.

One problem with imbalanced classification problems is that the machine learning model can have a difficult time predicting the minority classes because it sees far less examples. <br />
=> 클래스 불균형문제는 머신러닝 모델이 소수의 클래스들을 예측하는데 문제점이 발생하는 원인일 수 있습니다. 왜냐하면 그것에 대한 예제가 적기 때문입니다.

Think about this in human terms: if we are classifiying poverty and we see far more cases of no poverty than extreme poverty, it will make it more difficult for us to identify the high poverty households because of less exposure. [인간의 관점에서의 이해를 돕는 예시]

One potential method to address class imbalanceds is through oversampling (which is covered in more advanced notebooks).<br />
=> 따라서 이러한 클래스 불균형 문제를 해결하는 방법은 오버샘플링 이라는 방법이 존재합니다.

## Addressing Wrong Labels [잘못된 라벨 살펴보기]<br />
As with any realistic dataset, the Costa Rican Poverty data has some issues. <br />
=> 우리가 살펴볼 데이터셋에도 실제 데이터셋이 가지고 있는 문제와 같은 것이 존재합니다.<br />
Typically, 80% of a data science project will be spent cleaning data and fixing anomalies/errors. <br />
80%의 데이터 사이언스 프로젝트는 데이터를 정제하고 오류를 수정하는데 시간을 사용합니다.<br />
These can be either human entry errors, measurement errors, or sometimes just extreme values that are correct but stand out. <br />
=> 이것은 아마도 사람이 데이터를 넣는데 있어서 에러이거나, 측정오류이거나 또는 어떤 극한의 이상치인 경우가 있습니다.<br />
For this problem, some of the labels are not correct because individuals in the same household have a different poverty level. <br />
=> 이문제로 인해서, 몇개의 라벨은 같은 가구이지만 타겟 라벨이 다른 문제가 발생합니다.<br />
We're not told why this may be the case, but we are told to use the head of household as the true label.<br />
=> 이 컴피티션에서는 왜인지는 모르겠지만, 우리는 가구의 가장의 라벨이 맞다는 것을 계속 언급해 왔습니다.

That information makes our job much easier, but in a real-world problem, we would have to figure out the reason Why the labels are wrong and how to address the issue on our own. <br />
=> 이러한 정보는 우리가 하는 일을 더욱더 쉽게 하지만, 실제 영역에서는 우리는 왜 라벨들이 다른지에 대해서 찾아내야만 합니다.<br />
This section fixes the issue with the labels although it is not strictly necessary: I kept it in the notebook just to show how we may deal with this issue.<br />
=> 이렇게 라벨을 고치는 것이 필수는 아니지만 우리는 고쳐보려고합니다.

Identify Errors<br />
First we need to find the errors before we can correct them. <br />
일단 오류를 정정하기 전에 오류를 수집해봅시다.<br />
To find the households with different labels for family members, we can group the data by the household and then check if there is only one unique value of the Target.<br />
다른 라벨을 가진 가족을 찾아내기 위해서 우리는 데이터들을 가구별로 그룹을 짓고 타겟의 값이 고유한지 살펴보려고합니다.

In [ ]:
all_equal = train.groupby('idhogar')['Target'].apply(lambda x:x.nunique() == 1)

not_equal = all_equal[all_equal != True]
print('There are {} households where the family members do not all have the same target.'.format(len(not_equal)))

Let's look at one example.
=> 그렇다면 어떤 한 가족의 예시를 봅시다

In [ ]:
train[train['idhogar'] == not_equal.index[0]][['idhogar', 'parentesco1', 'Target']]

The organizers tell us that the correct label is that for the head of household, where parentesco1 == 1. <br />
=> 대회의 주최자는 가장의 라벨은 정확하다고 언급했습니다..<br />
For this household, the correct label is 3 for all members. <br />
=> 그래서 위의 경우에는 모든 가족들에게 3이라는 값을 할당하는 것이 옳습니다.<br />
We can correct this (as shown later) by reassigning all the individuals in this household the correct poverty level.<br />
=> 우리는 가장의 라벨 값을 잘못 개재되어 있는 라벨에 넣어 줄겁니다.<br />
In the real-world, you might have to make the tough decision of how to address the problem by yourself (or with the help of your team).<br />
=> 실제로는, 어떻게 이러한 문제를 해결할지 고민을 해봐야 할 것입니다.

## Families without Heads of Household [가장이 없는 가구]<br />
We can correct all the label discrepancies by assigning the individuals in the same household the label of the head of household. <br />
=> 우리는 이러한 타겟 라벨 불일치를 가장의 타겟 라벨을 할당해 줌으로써 옳게 만들어 줄 수 있습니다<br />
But wait, you may ask: "What if there are households without a head of household? And what if the members of those households have differing values of the label?"<br />
=> 그러나 간단한 질문을 해봅시다 "만약 가장이 없는 가구가 있다면? 그리고 그 구성원들이 다른 타겟 라벨을 가지고 있다면?"<br />
Well, since you asked, let's investigate exactly that question!<br />
=> 음... 그렇다면 자세히 이 질문에 대해서 알아보도록 합시다

In [ ]:
household_leader = train.groupby('idhogar')['parentesco1'].sum()

household_nohead = train.loc[train['idhogar'].isin(household_leader[household_leader==0].index),:]
print('There are {} households without a head.'.format(household_nohead['idhogar'].nunique()))

In [ ]:
household_nohead_equal = household_nohead.groupby('idhogar')['Target'].apply(lambda x:x.nunique() == 1)
print('{} Households with no head have different labels.'.format(sum(household_nohead_equal == False)))

Well that's a relief! <br />
=> 다행입니다<br />
This means that we don't have to worry about a household both where there is no head AND the members have different values of the label! <br />
=> 위를 통해 우리가 우려할 뻔 했던 두 가지 상황이 동시에 일어나는 경우는 없다는 것을 파악했습니다.<br />
For this problem, according to the organizers, if a household does not have a head, then there is no true label. <br />
=> 이 문제 때문에, 컴피티션의 계획자는 가장이 없는 가구는 제대로된 라벨을 가지고 있지 않다고 언급했습니다.<br />
Therefore, we actually won't use any of the households without a head for training Nonetheless, it's still a good exercise to go through this process of investigating the data!<br />
=> 그러므로 우리는 가장없는 가구의 구성원들을 사용하지 않을 것입니다. 

## Correct Errors [오류수정]<br />
Now we can correct labels for the households that do have a head AND the members have different poverty levels.<br />
=> 이제 우리는 각각의 가구의 타겟 라벨을 가장의 타겟라벨로 바꿔주도록 해봅시다.

In [ ]:
for household in not_equal.index:
    true_label = int(train.loc[(train['idhogar'] == household) & (train['parentesco1'] == 1)]['Target'])
    train.loc[train['idhogar'] == household,'Target'] = true_label
    
all_equal = train.groupby('idhogar')['Target'].apply(lambda x:x.nunique() == 1)
not_equal = all_equal[all_equal != True]

print('There are {} households where the family members do not all have the same target.'.format(len(not_equal)))

Since we are only going to use the heads of household for the labels, this step is not completely necessary but it shows a workflow for correcting data errors like you may encounter in real life. Don't consider it extra work, just practice for your career!<br />
=> 우리가 가장의 라벨만 사용하겠다고 했기 때문에, 이 과정은 별로 불필요해보이지만 기 과정은 실제 데이터셋에서 만날수 있는 에러를 고치는 방안중 하나입니다. 그렇기 때문에 귀찮은 일을 했다고 생각하지 말아줬으면 합니다.

## Missing Variables [손실 값]<br />
One of the most important steps of exploratory data analysis is finding missing values in the data and determining how to handle them. <br />
=> EDA에서 가장 중요한 과정은 손실 값들을 찾아내고 이를 어떻게 다룰지 결정하는 것입니다.

Missing values have to be filled in before we use a machine learning model and we need to think of the best strategy for filling them in based on the feature: this is where we'll have to start digging into the data definitions.<br />
=> 손실 값은 머신러닝 모델을 이용하기 전에 채워져야 합니다. 그리고 이를 채우는데 적절한 전략을 생각해봐야합니다. 이는 우리가 데이터의 정의를 살펴봐야하는 곳입니다.<br />

First we can look at the percentage of missing values in each column.<br />
=> 일단은 각각의 컬럼의 손실 값들의 비율을 계산해 봅시다.

In [ ]:
missing = pd.DataFrame(data.isnull().sum()).rename(columns={0:'total'})
missing['percent'] = missing['total'] / len(data)
missing.sort_values(by='percent',ascending=False).head(10).drop('Target')

We don't have to worry about the Target becuase we made that NaN for the test data. However, we do need to address the other 3 columns with a high percentage of missing values.<br />
=> 타겟값에 있어서 손실 값은 고려하지 않아도 됩니다. 왜냐하면 우리는 테스트셋에 대해서는 임의로 손실 값을 만드어 주었기 때문이죠. 그러나 3개의 높은 비율의 손실 값을 가진 컬럼을 찾아냈습니다.

v18q1: Number of tablets

Let's start with v18q1 which indicates the number of tablets owned by a family. <br />
=> 가족이 소유한 tablet의 수를 나타내는 컬럼에 대해서 알아 봅시다<br />
We can look at the value counts of this variable. 

Since this is a household variable, it only makes sense to look at it on a household level, so we'll only select the rows for the head of household.<br />
=> 이것은 가구별 변수이기 대문에, 가구당 레벨에서 이해를 하는 것이 옳아보입니다. 그래서 우리는 가장에 대해서만 알아보도록 합시다

## Function to Plot Value Counts [해당 컬럼의 값의 수를 시각화]<br />
Since we might want to plot value counts for different columns, we can write a simple function that will do it for us!

In [ ]:
def plot_value_counts(df,col,head_only=False):
    if head_only:
        df = df.loc[df['parentesco1']==1,col]
    plt.figure(figsize=[8,6])
    df[col].value_counts().sort_index().plot(kind='bar',color='blue',edgecolor='k',linewidth=2)
    plt.xlabel(f'{col}')
    plt.title(f'{col} Value Counts')
    plt.ylabel('Count')
    plt.show()

In [ ]:
plot_value_counts(heads,'v18q1')

It looks like the most common number of tablets to own is 1 if we go only by the data that is present. <br />
=> 존재하는 데이터만 고려했을 때 1대의 테블릿을 보유한 숫자가 가장 많아 보입니다.<br />
However, we also need to think about the data that is missing. <br />
=> 그러나 데이터가 손실 되었음을 염두할 필요가 있습니다.<br />
In this case, it could be that families with a nan in this category just do not own a tablet! <br />
=> 그래서, 해당 수치가 손실된 가족의 경우에는 테블릿을 가지고 있지 않을 것이라고 추측됩니다.<br />
If we look at the data definitions, we see that v18q indicates whether or not a family owns a tablet. <br />
=>v18q컬럼을 보게되면 해당 컬럼은 해당가족이 테블릿을 가지고 있는지 없는지 여부를 나타냅니다<br />
We should investigate this column combined with the number of tablets to see if our hypothesis holds.<br />
=> 그래서 우리의 가정이 맞는지 확인하기 위해서 해당 컬럼을 같이 알아볼 필요가 있을 듯합니다.

We can groupby the value of v18q (which is 1 for owns a tablet and 0 for does not) and then calculate the number of null values for v18q1. <br />
=> 우리는 그 컬럼을 그룹할 수 있습니다.<br />
This will tell us if the null values represent that the family does not own a tablet.<br />
=> 이 값이 null인 경우는 가족이 테블릿을 가지고 있지 않음을 알 수 있습니다.

In [ ]:
heads.groupby('v18q')['v18q1'].apply(lambda x:x.isnull().sum())

Well, that solves the issue! Every family that has nan for v18q1 does not own a tablet. Therefore, we can fill in this missing value with zero.<br />
=> v18q1에서 테블릿을 가지지 못한 모든 가족들이 nan값을 가짐을 찾아냈습니다. 그래서 우리는 빈 값을 0으로 채워줄 것입니다.

In [ ]:
data['v18q1'] = data['v18q1'].fillna(0)

## v2a1: Monthly rent payment<br />
=> 매달 지불하는 월세

The next missing column is v2a1 which represents the montly rent payment.<br />
=> 해당 컬럼은 매달 지불하는 월세의 정도를 나타냅니다

In addition to looking at the missing values of the monthly rent payment, it will be interesting to also look at the distribution of tipovivi_, the columns showing the ownership/renting status of the home. <br />
=> 이 컬럼의 손실 값을 보는데 있어서, tipoviv_(집의 소유/대여 구분을 타나내는 컬럼)을 참고해봐야 할 것 같습니다<br />
For this plot, we show the ownership status of those homes with a nan for the monthyl rent payment.<br />
=> 이 그림에서는 우리는 매달 내는 월세가 nan인 집의 소유상태를 보려고 합니다

In [ ]:
own_variables = [x for x in data if x.startswith('tipo')]

data.loc[data['v2a1'].isnull(),own_variables].sum().plot(kind='bar',figsize=[10,8],color='green',edgecolor='k',linewidth=2)

plt.xticks([0,1,2,3,4], ['Owns and Paid Off', 'Owns and Paying', 'Rented', 'Precarious', 'Other'], rotation = 60)
plt.title('Home Ownership Status for Households Missing Rent Payments', size = 18);

The meaning of the home ownership variables is below:<br />
=> 해당 갑의 정보는 아래와 같습니다

tipovivi1, =1 own and fully paid house<br />
tipovivi2, "=1 own,  paying in installments"<br />
tipovivi3, =1 rented<br />
tipovivi4, =1 precarious<br />
tipovivi5, "=1 other(assigned,  borrowed)"

We've solved the issue! Well, mostly: the households that do not have a monthly rent payment generally own their own home. <br />
=> 우리는 또다른 문제를 해결했습니다. 매달 내는 월세의 값이 없는 이유는 대부분 이미 그들 소유의 집에 살고 있기 때문입니다<br />
In a few other situations, we are not sure of the reason for the missing information.<br />
=> 몇몇 다른 상황들에 대해서는 왜 손실 값을 가지는지는 확신할 수 없을 것 같습니다

For the houses that are owned and have a missing monthly rent payment, we can set the value of the rent payment to zero. <br />
=>그래서 해당하는 경우의 월세값은 0으로 설정 할 수 있을 것 같습니다<br />
For the other homes, we can leave the missing values to be imputed but we'll add a flag (Boolean) column indicating that these households had missing values.<br />
=> 다른 집들의 경우에는 손실 값으로 남겨두지만, 하나의 논리형 컬럼을 이용하여 손실 값을 가지고 있음을 표시해둘 것입니다.

In [ ]:
data.loc[data['tipovivi1'] == 1,'v2a1'] = 0

data['v2a1-missing'] = data['v2a1'].isnull()
data['v2a1-missing'].value_counts()

## rez_esc: years behind in school<br />
=> 학교를 떠난 후의 년수

The last column with a high percentage of missing values is rez_esc indicating years behind in school.<br />
For the families with a null value, is possible that they have no children currently in school.<br />
=> 손실 값을 가지고 있는 가족들에 대하여, 최근에 학교에 진학한 아이들이 없을 수 있다<br />
Let's test this out by finding the ages of those who have a missing value in this column and the ages of those who do not have a missing value.<br />
=> 그래서 해당 컬럼의 손실값을 가지고있는 사람들의 연령과 손실값을 가지고 있지 않는 사람들의 연령을 비교해보자

In [ ]:
data.loc[data['rez_esc'].notnull(),'age'].describe()

What this tells us is that the oldest age with a missing value is 17.

For anyone older than this, maybe we can assume that they are simply not in school. Let's look at the ages of those who have a missing value.<br />
=> 17살보다 나이가 많은 사람들에 대해, 우리는 학교에 다니고 있지 않다고 가정할 수 있을 것 같습니다. 그렇다면 손실값을 가지고 있는 사람들에 대해서 알아볼까요?

In [ ]:
data.loc[data['rez_esc'].isnull(),'age'].describe()

If we read through some of the discussions for this competition, we learn that this variable is only defined for individuals between 7 and 19. <br />
=> 만약 이 컴피티션에 대한 디스커션을 읽어봤다면, 우리는 이 값이 7살과 19살 사이에 사람들에게만 있음을 알 수 있습니다.<br />
Anyone younger or older than this range presumably has no years behind and therefore the value should be set to 0. <br />
=> 그래서 이 연령에 해당하지 못하는 사람들에게는 0의 값을 부여해야 할 것 같습니다.<br />
For this variable, if the individual is over 19 and they have a missing value, or if they are younger than 7 and have a missing value we can set it to zero. <br />
For anyone else, we'll leave the value to be imputed and add a boolean flag.<br />
=> 그래서 그 연령에 해당하지 않는 사람들의 경우에는 논리형 컬럼으로 남겨둘 것입니다.

In [ ]:
# If individual is over 19 or younger than 7 and missing years behind, set it to 0
data.loc[((data['age'] > 19) | (data['age'] < 7)) & (data['rez_esc'].isnull()), 'rez_esc'] = 0

# Add a flag for those between 7 and 19 with a missing value
data['rez_esc-missing'] = data['rez_esc'].isnull()

There is also one outlier in the rez_esc column. <br />
=> rez_esc 컬럼에 하나의 이상치를 찾아 볼 수 있습니다.<br />
Again, if we read through the competition discussions, we learn that the maximum value for this variable is 5. Therefore, any values above 5 should be set to 5.<br />
=> 디스커션을 읽었다면 해당 값의 최대치는 5입니다. 따라서 5의 값을 넘는 것들에게 모두 5를 할당해 줍시다

In [ ]:
data.loc[data['rez_esc']>5,'rez_esc'] = 5

## Plot Two Categorical Variables [두개의 카테고리컬한 변수에 대한 시각화]<br />
To show how two categorical variables interact with one another, there are a number of plotting options: scatterplots, faceted bar plots, boxplots, etc. <br />
=> 두 개의 카테고리컬한 변수들에 대해서 시각화하는 방법에는: 산포도, 박스그림 등이 있습니다<br />
I wasn't satisfied with any of these choices so I wrote the function below, which essentially is a scatterplot of two categoricals where the size of the points represent the percentage of a given y-value represented by each x-value.
=>

In [ ]:
def plot_categoricals(x, y, data, annotate = True):
    """Plot counts of two categoricals.
    Size is raw count for each grouping.
    Percentages are for a given value of y."""
    
    # Raw counts 
    raw_counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = False))
    raw_counts = raw_counts.rename(columns = {x: 'raw_count'})
    
    # Calculate counts for each group of x and y
    counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = True))
    
    # Rename the column and reset the index
    counts = counts.rename(columns = {x: 'normalized_count'}).reset_index()
    counts['percent'] = 100 * counts['normalized_count']
    
    # Add the raw count
    counts['raw_count'] = list(raw_counts['raw_count'])
    
    plt.figure(figsize = (14, 10))
    # Scatter plot sized by percent
    plt.scatter(counts[x], counts[y], edgecolor = 'k', color = 'lightgreen',
                s = 100 * np.sqrt(counts['raw_count']), marker = 'o',
                alpha = 0.6, linewidth = 1.5)
    
    if annotate:
        # Annotate the plot with text
        for i, row in counts.iterrows():
            # Put text with appropriate offsets
            plt.annotate(xy = (row[x] - (1 / counts[x].nunique()), 
                               row[y] - (0.15 / counts[y].nunique())),
                         color = 'navy',
                         s = f"{round(row['percent'], 1)}%")
        
    # Set tick marks
    plt.yticks(counts[y].unique())
    plt.xticks(counts[x].unique())
    
    # Transform min and max to evenly space in square root domain
    sqr_min = int(np.sqrt(raw_counts['raw_count'].min()))
    sqr_max = int(np.sqrt(raw_counts['raw_count'].max()))
    
    # 5 sizes for legend
    msizes = list(range(sqr_min, sqr_max,
                        int(( sqr_max - sqr_min) / 5)))
    markers = []
    
    # Markers for legend
    for size in msizes:
        markers.append(plt.scatter([], [], s = 100 * size, 
                                   label = f'{int(round(np.square(size) / 100) * 100)}', 
                                   color = 'lightgreen',
                                   alpha = 0.6, edgecolor = 'k', linewidth = 1.5))
        
    # Legend and formatting
    plt.legend(handles = markers, title = 'Counts',
               labelspacing = 3, handletextpad = 2,
               fontsize = 16,
               loc = (1.10, 0.19))
    
    plt.annotate(f'* Size represents raw count while % is for a given y value.',
                 xy = (0, 1), xycoords = 'figure points', size = 10)
    
    # Adjust axes limits
    plt.xlim((counts[x].min() - (6 / counts[x].nunique()), 
              counts[x].max() + (6 / counts[x].nunique())))
    plt.ylim((counts[y].min() - (4 / counts[y].nunique()), 
              counts[y].max() + (4 / counts[y].nunique())))
    plt.grid(None)
    plt.xlabel(f"{x}"); plt.ylabel(f"{y}"); plt.title(f"{y} vs {x}");

In [ ]:
plot_categoricals('rez_esc', 'Target', data);

The size of the markers represents the raw count. <br />
=> 하나의 점의 크기는 개수를 뜻합니다<br />
To read the plot, choose a given y-value and then read across the row. <br />
=> 표를 읽으려면, 임의의 세로축 값을 고르고 가로로 하나씩 읽으면 됩니다.<br />
For example, with a poverty level of 1, 93% of individuals have no years behind with a total count of around 800 individuals and about 0.4% of individuals are 5 years behind with about 50 total individuals in this category.<br />
=> 예를들면 빈곤단계가 1인경우에는, 93퍼센트의 개개인이 years behind가 0이며 이 크기는 대략 800명 정도 된다는 것을 의미합니다.<br />
This plot attempts to show both the overall counts and the within category proportion; it's not perfect , but I gave it a shot!

In [ ]:
plot_categoricals('escolari', 'Target', data, annotate = False)

The remaining missing values in each column will be filled in, a process known as Imputation. <br />
=> 남은 각 컬럼의 손실 값들은 Imputation이라고 거치는 손실 값을 채우는 과정을 거치게 될 것입니다.<br />
There are several types of imputation commonly used, and one of the simplest and most effective methods is to fill in the missing values with the median of the column.<br />
=> 몇몇의 imputation 타입은 주로 사용되며, 그중 가장 간단하고 효과적인 방법은 중앙값을 넣는 것입니다.

As a final step with the missing values, we can plot the distribution of target for the case where either of these values are missing.<br />
=> 손실값의 마지막 단게로서, 우리는 손실값들의 타겟 라벨의 분포를 그려볼 것입니다.

In [ ]:
plot_value_counts(data[(data['rez_esc-missing'] == 1)], 'Target')

In [ ]:
plot_value_counts(data[(data['v2a1-missing'] == 1)], 'Target')

This looks like it could be an indicator of more poverty given the higher prevalence of 2: moderate poverty.

This represents an important point: sometimes the missing information is just as important as the information you are given.<br />
=> 가끔은 손실 값들이 중요한 정보를 가지고 있다는 것을 보여주기도 합니다.

## Feature Engineering

There is plenty more exploratory data analysis we can do, but first we should work on consolidating our data at a household level. <br />
=> 우리가 할 수 있는 EDA는 굉장히 많지만, 우리의 데이터들을 가구당 단위로 합병하는 작업을 진행해야합니다.<br />
We already have some of the information for each household, but for training, we will need all of the information summarized for each household. <br />
=> 우리는 각각의 가구별 정보를 조금 가지고 있지만, 훈련을 위해서는 각각의 가구를 위해 정보를 간략화할 필요가 있습니다.<br />
This means grouping the individuals in a house (groupby) and performing an aggregation (agg) of the individual variables.<br />
=> 이는 개개인을 가구별로 그룹을 짓고 개별 값을 병합해야합니다.

In another notebook, I show how we can use automated feature engineering to do this, and automated feature engineering should be a standard part of the machine learning workflow. <br />
=> 필자는 feature engineering에 있어서 자동화된 방법을하려고 하며, 이러한 방법은 머신러닝 작업 흐름에서 굉장히 기본적인 부분입니다<br />
Right now, we'll stick to doing this by hand, but definitely take a look at automated feature engineering in Featuretools.<br />
=> 우리는 수작업으로 할 것이지만, 자동적으로 하는 방법을 꼭 참고해보시길 바랍니다

## Column Definitions [컬럼의 의미]<br />
Sometimes in data science we have to get our hands dirty digging through the data or do tedious tasks that take a lot of time. <br />
=> 데이터 사이언스에서 우리는 직접 데이터에서 무언가를 찾아내야하고 시간이 많이걸리는 지루한 작업들을 해야할 수 있습니다.<br />
This is that part of the analysis: we have to define the columns that are at an individual level and at a household level using the data decsriptions. <br />
=> 이것은 분석의 단계입니다: 각각의 컬럼을 정의하고 어느 것이 개개인의 수준인가 또는 가구별 수준인지 설명을 통해 알아내야합니다<br />
There is simply no other way to identify which variables at are the household level than to go through the variables themselves in the data description. <br />
=> 데이터 설명을 보지 않고서는 이것을 밝혀낼만한 다른 방법은 없을 것입니다.<br />
Except, I've already done this for you, so all you have to do is copy and paste!<br />
=> 필자는 이미 작업을 해두었으니, 해야할 일은 가져다 쓰는 것입니다.

We'll define different variables because we need to treat some of them in a different manner. <br />
=> 다른 방법으로 몇몇이 사용되기 때문에 이들을 위해 몇몇의 다른 변수를 선언할 것입니다<br />
Once we have the variables defined on each level, we can work to start aggregating them as needed.<br />
=> 각각의 레벨에서 쓰여질 변수들이 정의되고 난 뒤에, 이를 사용하기 위해서 병합을 시작할 것입니다.

The process is as follows<br />
=> 해당 과정은 아래의 과정과 같습니다.

1. Break variables into household level and invididual level<br />
=> 변수를 가구단위 와 개별단위로 구분<br />
2. Find suitable aggregations for the individual level data<br />
=> 개개인 수준의 데이터를 위해서 적절한 병합방법을 찾습니다<br />
    Ordinal variables can use statistical aggregations<br />
    => Ordinal 값들은 통계적 병합에 사용될 것이며<br />
    Boolean variables can also be aggregated but with fewer stats<br />
    => 논리형 자료들은 적은 통계치들에 병합될 것이며<br />
3. Join the individual aggregations to the household level data<br />
=> 개인 수준의 데이터를 가구별 수준의 데이터에 합치고


## Define Variable Categories[변수의 카테고리들을 정의]

There are several different categories of variables:

1.Individual Variables: these are characteristics of each individual rather than the household<br />
    Boolean: Yes or No (0 or 1)<br />
    Ordered Discrete: Integers with an ordering<br />
2. Household variables<br />
    Boolean: Yes or No<br />
    Ordered Discrete: Integers with an ordering<br />
    Continuous numeric<br />
3. Squared Variables: derived from squaring variables in the data<br />
4. Id variables: identifies the data and should not be used as features

Below we manually define the variables in each category. This is a little tedious, but also necessary.<br />
=> 아래에 우리는 각각의 카테고리의 변수들을 정의해두었습니다.

In [ ]:
id_ = ['Id', 'idhogar', 'Target']

In [ ]:
ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone', 'rez_esc-missing']

ind_ordered = ['rez_esc', 'escolari', 'age']

In [ ]:
hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2', 'v2a1-missing']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding']

In [ ]:
sqr_ = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 
        'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq']

Let's make sure we covered all of the variables and didn't repeat any.<br />
=> 혹시 반복되는 것은 없는지 확인해봅시다.

In [ ]:
x = ind_bool + ind_ordered + id_ + hh_bool + hh_ordered + hh_cont + sqr_

from collections import Counter

print('There are no repeats: ', np.all(np.array(list(Counter(x).values())) == 1))
print('We covered every variable: ', len(x) == data.shape[1])

In [ ]:
np.all(np.array(list(Counter(x).values())) == 1)

## Squared Variables [제곱된 값들] <br />
First, the easiest step: we'll remove all of the squared variables.<br />
=> 가장 쉬운 방법으로, 우리는 모든 이와 같은 값들을 지울 수 있습니다.<br />
Sometimes variables are squared or transformed as part of feature engineering because it can help linear models learn relationships that are non-linear. <br />
=> 그러나 가끔씩 우리는 이러한 성격을 지닌 값들이 비선형관계를 선형모델이 학습을 하도록 도와주기 때문에 사용합니다.<br />
However, since we will be using more complex models, these squared features are redundant. <br />
=> 그러나 더욱 더 복잡한 모델을 사용할 것이기 때문에, 위와 같은 특징들은 중복이 됩니다.<br />
They are highly correlated with the non-squared version, and hence can actually hurt our model by adding irrelevant information and also slowing down training.<br />
=> 이 값들은 제곱이 취해지기 전의 컬럼과 굉장히 높은 상관관계를 보이며, 이는 더 나아가 우리의 모델에 이롭지 않을 수 있습니다.<br />

For an example, let's take a look at SQBage vs age<br />
=> 예시로 알아봅시다

In [ ]:
sns.lmplot('age','SQBage',data=data,fit_reg=False)
plt.title('Squared Age versus Age')

These variables are highly correlated, and we don't need to keep both in our data.<br />
=> 위와 같이 이러한 성격의 값은 괴앚ㅇ히 상관관계가 커서 둘 다 모두를 데이터셋에 남겨둘 필요가 없습니다.

In [ ]:
data = data.drop(columns = sqr_)
data.shape

## Id Variables<br />
These are pretty simple: they will be kept as is in the data since we need them for identification.<br />
=> Id 값은 각각을 식별하기 위해 반드시 필요하므로 삭제등을 하지 않습니다.

## Household Level Variables<br />
First let's subset to the heads of household and then to the household level variables.<br />
=> 가장들의 서브셋을 만들고 가구별 컬럼들을 추출합니다.

In [ ]:
heads = data.loc[data['parentesco1']== 1,:]
heads = heads[id_ + hh_bool + hh_cont + hh_ordered]
heads.shape

For most of the household level variables, we can simply keep them as is: since we want to make predictions for each household, we use these variables as features. <br />
=> 대부분의 가구별 수준의 변수들에 대해, 우리가 가구별로 예측을 하려고 할 때에는 위의 변수들을 사용합니다<br />
However, we can also remove some redundant variables and also add in some more features derived from existing data.<br />
=> 그러나 이 데이터들에서도 중복되는 값들을 제거해야 하며 며쳧의 특징들은 존재하는 데이터로부터 파생된 경우가 있습니다.<br />

Redundant Household Variables [중복된 가구단위의 변수]

Let's take a look at the correlations between all of the household variables. <br />
=> 가구단위의 변수들의 상관관게를 알아보도록 합시다<br />
If there are any that are too highly correlated, then we might want to remove one of the pair of highly correlated variables.<br />
=>만약 너무 많이 높은 상관관계를 보이는 것이 있다면, 우리는 그 쌍중 하나를 삭제할 필요가 있습니다.<br />

The following code identifies any variables with a greater than 0.95 absolute magnitude correlation.<br />
=> 다음의 코드는 절대값 0.95이상의 상관관게를 가지는 변수들을 알아내는 코드입니다.

In [ ]:
corr_matrix = heads.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

to_drop = [col for col in upper.columns if any(abs(upper[col]) > 0.95)]

to_drop

These show one out of each pair of correlated variables. To find the other pair, we can subset the corr_matrix.<br />
=> 관련된 변수의 각각의 쌍중 하나를 보여줌. 다른 쌍을 찾기위해서는 corr_matrix를 만든다

In [ ]:
corr_matrix.loc[corr_matrix['tamhog'].abs()>0.9,corr_matrix['tamhog'].abs()>0.9]

In [ ]:
sns.heatmap(corr_matrix.loc[corr_matrix['tamhog'].abs() > 0.9, corr_matrix['tamhog'].abs() > 0.9],annot=True, cmap = plt.cm.autumn_r, fmt='.3f');

"tamhog는 가구의 사이즈를 뜻함"<br />

There are several variables here having to do with the size of the house:<br />
=> 집의 사이즈와 관련 있는 변수들은 아래와 같습니다.

r4t3, Total persons in the household [집에 살고 있는 총인원 수]<br />
tamhog, size of the household [가구의 크기]<br />
tamviv, number of persons living in the household [가구에 살고 있는 사람의 수]<br />
hhsize, household size [가구의 크기]<br />
hogar_total, # of total individuals in the household [가구에 있는 개별 인원의 총합]<br />

These variables are all highly correlated with one another. <br />
=> 이러한 값들은 서로 큰 연관성을 가지고 있습니다.<br />
In fact, hhsize has a perfect correlation with tamhog and hogar_total. <br />
=> 실제로 hhsize는 tamhog나 hogar_total과 완벽한 선형성을 보여줍니다<br />
We will remove these two variables because the information is redundant. We can also remove r4t3 because it has a near perfect correlation with hhsize.<br />
=> 이 정보는 중복이기 때문에 두 변수를 제거해 줄 것입니다. r4t3또한 거의 완벽한 중복에 가깝기 때문에 지워줄 것입니다.

tamviv is not necessarily the same as hhsize because there might be family members that are not living in the household. Let's visualize this difference in a scatterplot.<br />
=>tamviv는 hhsize와 필수적으로 같을 필요는 없습니다. 왜냐하면 가구에 살고 있지 않는 가족들도 있기 때문이죠

In [ ]:
heads = heads.drop(columns = ['tamhog','hogar_total','r4t3'])

In [ ]:
sns.lmplot('tamviv','hhsize',data,fit_reg=False,size=8)
plt.title('Household size vs number of persons living in the household');

We see for a number of cases, there are more people living in the household than there are in the family. <br />
=> 우리는 다양한 케이스들을 봤고, 가족의 수보다 가구에 살고 있는 사람들이 많습니다<br />
This gives us a good idea for a new feature: the difference between these two measurements!<br />
=> 이는 우리가 두 인원의 차이를 새로운 feature로 쓰는 인사이트를 제공합니다.

Let's make this new feature.<br />
=> 새로운 feature를 만들어 봅시다.

In [ ]:
heads['hhsize-diff'] = heads['tamviv'] - heads['hhsize']
plot_categoricals('hhsize-diff','Target',heads)

Even though most households do not have a difference, there are a few that have more people living in the household than are members of the household.<br />
=> 비록 많은 가구들이 차이를 가지고 있지 않더라도, 몇몇의 차이가 나는 가구들이 존재합니다.

Let's move on to the other redundant variables. First we can look at coopele<br />
=> 이제는 다른 변수로 넘어가 봅시다

In [ ]:
corr_matrix.loc[corr_matrix['coopele'].abs() > 0.9,corr_matrix['coopele'].abs() > 0.9]

These variables indicate where the electricity in the home is coming from. <br />
=> 이 변수들은 집의 전기가 어디서 오는지를 가르킵니다.<br />
There are four options, and the families that don't have one of these two options either have no electricity (noelec) or get it from a private plant (planpri).<br />
=> 여기에는 네가지 옵션이 있습니다. 그리고 전기가 없거나 개인 발전을 하는 선택지를 동시에 선택할 수 없습니다.

## Creating Ordinal Variable<br />
I'm going to compress these four variables into one by creating an ordinal variable. I'm going to choose the mapping myself, based on the data decriptions:<br />
=> 네가지 컬럼을 하나로 합칠 것입니다. 합치는 데 있어서 매핑은 아래와 같습니다.

0: No electricity<br />
1: Electricity from cooperative<br />
2: Electricity from CNFL, ICA, ESPH/JASEC<br />
3: Electricity from private plant<br />

An ordered variable has an inherent ordering, and for this we choose our own based on the domain knowledge. <br />
=> <br />
After we create this new ordered variable, we can drop the four others. <br />
=> 이를 통해 하나의 컬럼을 만들면 다른 컬럼들을 없애줍니다<br />
There are several households that do not have a variable here, so we will use a nan (which will be filled in during imputation) and add a Boolean column indicating there was no measure for this variable.<br />
=> 만약 값이 없다면 나중에 채워줄 것이므로 해당하는 값을 구분하기 위해서 논리형 컬럼을 추가하여줍니다

In [ ]:
elec = []

for i,row in heads.iterrows():
    if row['noelec'] == 1:
        elec.append(0)
    elif row['coopele'] == 1:
        elec.append(1)
    elif row['public'] == 1:
        elec.append(2)
    elif row['planpri'] == 1:
        elec.append(3)
    else:
        elec.append(np.nan)
        
heads['elec'] = elec
heads['elec-missing'] = heads['elec'].isnull()

heads = heads.drop(columns = ['noelec','coopele','public','planpri'])

In [ ]:
plot_categoricals('elec','Target',heads)

We can see that for every value of the Target, the most common source of electricity is from one of the listed providers.<br />
=> 그림에서도 볼 수 있듯이 가장 평범한 방법은 공식적인 공급자에게 받는 것입니다.

The final redundant column is area2. <br />
=> 마지막 컬럼은 area2입니다.<br />
This means the house is in a rural zone, but it's redundant because we have a column indicating if the house is in a urban zone. <br />
=> 이 컬럼은 집이 교외에 있는지를 물어보는 것입니다. 그러나 이미 집이 도심에 있는지에 대한 컬럼이 있기 때문에 해당 컬럼은 중복되었다고 할 수 있습니다.<br />
Therefore, we can drop this column.<br />
=> 그러므로 해당 컬럼을 제거합니다.

In [ ]:
corr_matrix.loc[corr_matrix['area2'].abs() >0.9, corr_matrix['area2'].abs()>0.9]

In [ ]:
heads = heads.drop(columns='area2')

heads.groupby('area1')['Target'].value_counts(normalize=True)

It seems like households in an urban area (value of 1) are more likely to have lower poverty levels than households in a rural area (value of 0).<br />
=> 위에 의하면 도심에 사는 가구가 취약계층들을 별로 가지고 있는 것을 알 수 있습니다.

## Creating Ordinal Variables <br />
For the walls, roof, and floor of the house, there are three columns each: the first indicating 'bad', the second 'regular', and the third 'good'. <br />
=> 집의 벽, 지붕, 바닥에 대해 세개의 컬럼이 있습니다. <br />
We could leave the variables as booleans, but to me it makes more sense to turn them into ordinal variables because there is an inherent order: bad < regular < good. <br />
=> 우리는 이들을 논리형으로 남길 수 있습니다. 하지만 필자는 이를 순서형으로 바꾸었으면 합니다 나쁨 보통 좋음 순으로<br />
To do this, we can simply find whichever column is non-zero for each household using np.argmax.

Once we have created the ordinal variable, we are able to drop the original variables.

In [ ]:
heads['walls'] = np.argmax(np.array(heads[['epared1','epared2','epared3']]),axis=1)
heads.drop(columns=['epared1','epared2','epared3'])

plot_categoricals('walls','Target',heads)

In [ ]:
# Roof ordinal variable
heads['roof'] = np.argmax(np.array(heads[['etecho1', 'etecho2', 'etecho3']]),
                           axis = 1)
heads = heads.drop(columns = ['etecho1', 'etecho2', 'etecho3'])

# Floor ordinal variable
heads['floor'] = np.argmax(np.array(heads[['eviv1', 'eviv2', 'eviv3']]),
                           axis = 1)
heads = heads.drop(columns = ['eviv1', 'eviv2', 'eviv3'])

## Feature Construction<br />
In addition to mapping variables to ordinal features, we can also create entirely new features from the existing data, known as feature construction. <br />
=> 기존에 존재하는 데이터를 가지고 새로운 컬럼을 만들어 낼 수도 있습니다.<br />
For example, we can add up the previous three features we just created to get an overall measure of the quality of the house's structure.<br />
=> 예를들면, 우리가 만들었던 세가지의 컬럼을 결합한 컬럼을 만들 수 있습니다

In [ ]:
heads['wall+roof+floor'] = heads['walls']+heads['roof']+heads['floor']

plot_categoricals('wall+roof+floor','Target',heads,annotate=False)

This new feature may be useful because it seems like a Target of 4 (the lowest poverty level) tends to have higher values of the 'house quality' variable. <br />
=> 이는 생각보다 유용합니다. 왜냐하면 4 타겟 라벨을 가진 값들이 대체로 집의 품질이 좋다고 판단할 수 있기 때문입니다.<br />
We can also look at this in a table to get the fine-grained details.

In [ ]:
count = pd.DataFrame(heads.groupby('wall+roof+floor')['Target'].value_counts(normalize=True)).rename(columns={'Target':'Normalized Count'}).reset_index()
count.head()

## The next variable will be a warning about the quality of the house. <br />
=> 다음으로는 집의 위험도 정도를 나타내보자<br />
It will be a negative value, with -1 point each for no toilet, electricity, floor, water service, and ceiling.

In [ ]:
heads['warning'] = 1 * (heads['sanitario1'] + (heads['elec'] == 0) + heads['pisonotiene'] + heads['abastaguano'] + (heads['cielorazo'] == 0))

We can keep using our plot_categoricals function to visualize these relationships, but seaborn also has a number of plotting options that can work with categoricals. <br />
One is the violinplot which shows the distribution of a variable on the y axis with the width of each plot showing the number of observations in that category.

In [ ]:
plt.figure(figsize = (10, 6))
sns.violinplot(x = 'warning', y = 'Target', data = heads);
plt.title('Target vs Warning Variable');

In [ ]:
plot_categoricals('warning', 'Target', data = heads)

The violinplot is not great here because it smooths out the categorical variable with the effect that it looks as if the Target can take on lesser and greater values than in reality. <br />
=> 바이올린 플롯은 여기서는 그리 좋아보이지 않습니다. 왜냐하면 만약 타겟이 실제보다 작거나 크면 보이는데 카테고리 변수가 영향을 받기 때문입니다.<br />
Nonetheless, we can see a high concentration of households that have no warning signs and have the lowest level of poverty. <br />
=> 그럼에도 불구하고 우리는 많은 가구들이 위험요소가 적으며 빈곤여부가 적다는 것을 알 수 있습니다.<br />
It looks as if this may be a useful feature, but we can't know for sure until we get to modeling!<br />
=> 유용한 것 처럼 보이지만, 모델링때 쓰여질 지는 모르겠습니다.

The final household feature we can make for now is a bonus where a family gets a point for having a refrigerator, computer, tablet, or television.

In [ ]:
# Owns a refrigerator, computer, tablet, and television
heads['bonus'] = 1 * (heads['refrig'] + 
                      heads['computer'] + 
                      (heads['v18q1'] > 0) + 
                      heads['television'])

sns.violinplot('bonus', 'Target', data = heads,
                figsize = (10, 6));
plt.title('Target vs Bonus Variable');

Per Capita Features<br />
Additional features we can make calculate the number of certain measurements for each person in the household.

In [ ]:
heads['phones-per-capita'] = heads['qmobilephone'] / heads['tamviv']
heads['tablets-per-capita'] = heads['v18q1'] / heads['tamviv']
heads['rooms-per-capita'] = heads['rooms'] / heads['tamviv']
heads['rent-per-capita'] = heads['v2a1'] / heads['tamviv']

## Exploring Household Variables [가구별 변수]<br />
After going to all the trouble of getting our features in order, now we can take a look at them in relation to the Target. <br />
=> 순서를 가진 컬럼들의 문제들을 모두 해결 했으니 이제 타겟 라벨과의 관계를 살펴봅시다<br />
We've already done a little of this, but now we can try to quantify relationships.<br />
=> 이미 조금했지만, 양적인 관계를 ㅏㄹ펴봅시다

## Measuring Relationships<br />
There are many ways for measuring relationships between two variables. Here we will examine two of these:<br />
=> 관계를 측정하는 방법에는 여러방법이 있지만 우리는 그중 두가지만 채택합시다

   **The Pearson Correlation: from -1 to 1 measuring the linear relationship between two variables [일반적으로 사용되는 상관계수]<br />
   The Spearman Correlation: from -1 to 1 measuring the monotonic relationship between two variables [해당 지표가 점차적으로 증가 또는 감소하는 관게를 보여주는 지표]<br />**

The Spearman correlation is 1 if as one variable increases, the other does as well, even if the relationship is not linear. 

On the other hand, the Pearson correlation can only be one if the increase is exactly linear. These are best illustrated by example.

In [ ]:
from scipy.stats import spearmanr

def plot_corrs(x,y):
    spr = spearmanr(x,y).correlation
    pcr = np.corrcoef(x,y)[0,1]
    
    data = pd.DataFrame({'x':x,'y':y})
    plt.figure(figsize=[6,4])
    sns.regplot('x','y',data=data,fit_reg=False)
    plt.title(f'Spearman: {round(spr, 2)}; Pearson: {round(pcr, 2)}');

In [ ]:
x = np.array(range(100))
y = x ** 2

plot_corrs(x, y)

The Spearman correlation is often considered to be better for ordinal variables such as the Target or the years of education. <br />
=> Spearman 상관계수는 타겟이나 교육년수와 같은 순서적인 변수들에게 좋을 것이며<br />
Most relationshisp in the real world aren't linear, and although the Pearson correlation can be an approximation of how related two variables are, it's inexact and not the best method of comparison.<br />
실제로는 모든 관계가 선형일리는 없으며, 비록 Pearson 상관계수가 두 변수가 어떻게 관계있는지에 대한 대략적인 것을 보여 줄지라도 비교에 있어서 항상 최고의 방법은 아니다

In [ ]:
x = np.array([1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 7, 8, 8, 9, 9, 9])
y = np.array([1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 3, 3, 2, 4, 2, 2, 4])

plot_corrs(x, y)

In most cases, the values are very similar.

In [ ]:
x = np.array(range(-19, 20))
y = 2 * np.sin(x)

plot_corrs(x, y)

First, we'll calculate the Pearson correlation of every variable with the Target.

In [ ]:
train_heads = heads.loc[heads['Target'].notnull(),:].copy()

pcorrs = pd.DataFrame(train_heads.corr()['Target'].sort_values()).rename(columns = {'Target':'pcorr'}).reset_index()
pcorrs = pcorrs.rename(columns= {'index':'feature'})

print('Most negatively correlated variables:')
print(pcorrs.head())

print('\nMost positively correlated variables:')
print(pcorrs.dropna().tail())

For the negative correlations, as we increase the variable, the Target decreases indicating the poverty severity increases. <br />
=> 부의 상관관계는 임의의 해당 상관관게의 값이 증가하면 타겟라벨은 감소할 것입니다(이는 더 빈곤하다는 것을 의미함)<br />
Therefore, as the warning increases, the poverty level also increases which makes sense because this was meant to show potential bad signs about a house. <br />
=> 그러므로 warning이 증가하면 부의 타겟 라벨은 더 빈곤하다는 것을 의미할 것입니다..<br />
The hogar_nin is the number of children 0 - 19 in the family which also makes sense: younger children can be financial source of stress on a family leading to higher levels of poverty. <br />
=> hogar_nin은 0-19세의 아이들의 수인데 이 또한 말이 맞는듯합니다: 어린 아이들은 경제적인 지원이 많이 필요하기 때문에 가족의 부의 수준을 떨어뜨릴 것입니다.<br />
Or, families with lower socioeconomic status have more children in the hopes that one of them will be able to succeed.<br />
=> 또는 <br />
Whatever the explanation, there is a real link between family size and poverty<br />
=> 

On the other hand, for the positive correlations, a higher value means a higher value of Target indicating the poverty severity decreases. <br />
=> 반면 양의 관계에서는, 위의 부의 관계와 반대로 생각하시면 됩니다.<br />
The most highly correlated household level variable is meaneduc, the average education level of the adults in the household. <br />
=> 가장 높은 수치는 meaneduc이며, 이는 가정의 어른들의 평균 교육 단게를 의미합니다.<br />
This relationship between education and poverty intuitively makes sense: greater levels of education generally correlate with lower levels of poverty.<br />
=> 교육 수준과 부의 단계는 또한 말이 맞는 것 같습니다.<br />
We don't necessarily know which causes which, but we do know these tend to move in the same direction.<br />
=> 어느것이 타겟라벨을 움직이는지 면밀히 파악할 필요는 없지만, 양의 방향으로 같이 관계를 가진다는 정도로 알아둘 필요가 있습니다.

The general guidelines for correlation values are below, but these will change depending on who you ask (source for these):<br />
=> 아래는 상관관계를 해석하는 수치별 정도입니다.

.00-.19 “very weak”<br />
.20-.39 “weak”<br />
.40-.59 “moderate”<br />
.60-.79 “strong”<br />
.80-1.0 “very strong”<br />
What these correlations show is that there are some weak relationships that hopefully our model will be able to use to learn a mapping from the features to the Target.

Now we can move on to the Spearman correlation.

In [ ]:
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)

feats = []
scorr = []
pvalues = []

# Iterate through each column
for c in heads:
    # Only valid for numbers
    if heads[c].dtype != 'object':
        feats.append(c)
        
        # Calculate spearman correlation
        scorr.append(spearmanr(train_heads[c], train_heads['Target']).correlation)
        pvalues.append(spearmanr(train_heads[c], train_heads['Target']).pvalue)

scorrs = pd.DataFrame({'feature': feats, 'scorr': scorr, 'pvalue': pvalues}).sort_values('scorr')

The Spearman correlation coefficient calculation also comes with a pvalue indicating the significance level of the relationship. <br />
=> Spearman 상관계수는 관계의 유의수준을 결정하는 pvalue와 함께 계산합니다.<br />
Any pvalue less than 0.05 is genearally regarded as significant, although since we are doing multiple comparisons, we want to divide the p-value by the number of comparisons, a process known as the Bonferroni correction.<br />
=> 다중 비교를 하더라도 임의의 pvalue가 0.05보다 작은 경우 우리는 유의하다고 판단합니다. 그래서 비교 횟수별로 p-value를 나눕니다. 이 과정을 베루누이 상관관계라고 합니다.

In [ ]:
print('Most negative Spearman correlations:')
print(scorrs.head())
print('\nMost positive Spearman correlations:')
print(scorrs.dropna().tail())

For the most part, the two methods of calculating correlations are in agreement. Just out of curiousity, we can look for the values that are furthest apart.

In [ ]:
corrs = pcorrs.merge(scorrs, on = 'feature')
corrs['diff'] = corrs['pcorr'] - corrs['scorr']

corrs.sort_values('diff').head()

In [ ]:
corrs.sort_values('diff').dropna().tail()

The largest discrepancy in the correlations is dependency. We can make a scatterplot of the Target versus the dependency to visualize the relationship. We'll add a little jitter to the plot because these are both discrete variables.

In [ ]:
sns.lmplot('dependency', 'Target', fit_reg = True, data = train_heads, x_jitter=0.05, y_jitter=0.05);
plt.title('Target vs Dependency');

It's hard to see the relationship, but it's slightly negative: as the dependency increases, the value of the Target decreases. <br />
=> 상관관계를 보긴 어렵지만 약간의 부의 상관관게를 가짐을 의미합니다.<br />
This makes sense: the dependency is the number of dependent individuals divided by the number of non-dependents. <br />
=> 이는 dependency가 부양해야할 가족 / 부양하는 가족 을 의미하기 때문에 이를 유추할수 있습니다.<br />
As we increase this value, the poverty severty tends to increase: having more dependent family members (who usually are non-working) leads to higher levels of poverty because they must be supported by the non-dependent family members.

In [ ]:
sns.lmplot('rooms-per-capita', 'Target', fit_reg = True, data = train_heads, x_jitter=0.05, y_jitter=0.05);
plt.title('Target vs Rooms Per Capita');

Correlation Heatmap<br />
One of my favorite plots is the correlation heatmap because it shows a ton of info in one image. For the heatmap, we'll pick 7 variables and show the correlations between themselves and with the target.

In [ ]:
variables = ['Target', 'dependency', 'warning', 'wall+roof+floor', 'meaneduc',
             'floor', 'r4m1', 'overcrowding']

# Calculate the correlations
corr_mat = train_heads[variables].corr().round(2)

# Draw a correlation heatmap
plt.rcParams['font.size'] = 18
plt.figure(figsize = (12, 12))
sns.heatmap(corr_mat, vmin = -0.5, vmax = 0.8, center = 0, 
            cmap = plt.cm.RdYlGn_r, annot = True);

This plot shows us that there are a number of variables that have a weak correlation with the Target. There are also high correlations between some variables (such as floor and walls+roof+floor) which could pose an issue because of collinearity.

Features Plot
For the final exploration of the household level data, we can make a plot of some of the most correlated variables with the Target. This shows scatterplots on the upper triangle, kernel density estimate (kde) plots on the diagonal, and 2D KDE plots on the lower triangle.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plot_data = train_heads[['Target', 'dependency', 'wall+roof+floor','meaneduc', 'overcrowding']]

grid = sns.PairGrid(data=plot_data,size=4,diag_sharey=False,hue='Target',hue_order=[4,3,2,1],vars = [col for col in list(plot_data.columns) if col != 'Target'])

grid.map_upper(plt.scatter,alpha=0.8,s=20)

grid.map_diag(sns.kdeplot)

grid.map_lower(sns.kdeplot,cmap=plt.cm.OrRd_r)
grid = grid.add_legend()
plt.suptitle('Feature Plots Colored By Target', size = 32, y = 1.05);

We'll leave the feature engineering of the household variables for now. Later, we can come back to this step if we are not pleased with the model performance.

In [ ]:
household_feats = list(heads.columns)

Individual Level Variables [개별 단위 변수]<br />
There are two types of individual level variables: Boolean (1 or 0 for True or False) and ordinal (discrete values with a meaningful ordering).<br />
=> 개별 단위 변수에는 논리형과 순서형이 있습니다.

In [ ]:
ind = data[id_ + ind_bool + ind_ordered]
ind.shape

Redundant Individual Variables [개별 단위 변수에 대한 중복처리]<br />
We can do the same process we did with the household level variables to identify any redundant individual variables. <br />
=> 우리는 개별 단위 변수에도 유사한 방법을 적용하려고 합니다<br />
We'll focus on any variables that have an absolute magnitude of the correlation coefficient greater than 0.95.<br />
=> 상관계수가 0.95이상인 값들에 대해서만 확인해 봅시다

In [ ]:
corr_matrix = ind.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

to_drop = [col for col in upper.columns if any(abs(upper[col]) > 0.95)]

to_drop

This is simply the opposite of male! We can remove the male flag.

In [ ]:
ind = ind.drop(columns = 'male')

Creating Ordinal Variables [순서척도 만들기]<br />
Much as we did with the household level data, we can map existing columns to an ordinal variable.<br />
=> 가구단위에서 했듯이 논리형 컬럼들을 결합하여 하나의 순서척도 컬럼을 만들 수 있습니다.<br />
Here we will focus on the instlevel_ variables which indicate the amount of education an individual has from instlevel1: no level of education to instlevel9: postgraduate education.<br />
=> instlevel_ 변수들은 개개인의 교육의 정도를 나타내는데 1부터 9까지 존재합니다.

To create the ordinal variable, for each individual, we will simply find which column is non-zero. <br />
=> 순서척도화 하기위해 각 개별 변수들에 0이 아닌 컬럼들을 찾기만 하면 됩니다.<br />
The education has an inherent ordering (higher is better) so this conversion to an ordinal variable makes sense in the problem context.<br />
=> 교육수준은 높은 순서가 더 좋은 교육수준을 나타내도록 합시다

In [ ]:
ind[[col for col in ind.columns if col.startswith('instlevel')]].head()

In [ ]:
ind['inst'] = np.argmax(np.array(ind[[col for col in ind.columns if col.startswith('instlevel')]]),axis=1)

plot_categoricals('inst','Target',ind,annotate=False)

Higher levels of education seem to correspond to less extreme levels of poverty. <br />
=> 높은 교육수준은 타겟라벨이 높은 수준으로 상관관계를 가지며<br />
We do need to keep in mind this is on an individual level though and we eventually will have to aggregate this data at the household level.<br />
=> 우리는 개별적인 컬럼을 전체 가구수준의 컬럼으로 합치려는 작업을 해야 한다는 것을 명심해야한다.

In [ ]:
plt.figure(figsize = (10, 8))
sns.violinplot(x = 'Target', y = 'inst', data = ind);
plt.title('Education Distribution by Target');

In [ ]:
# Drop the education columns
ind = ind.drop(columns = [c for c in ind if c.startswith('instlevel')])
ind.shape

Feature Construction<br />
We can make a few features using the existing data. <br />
=> 존재하는 데이터를 이용해서 몇개의 컬럼들을 만들어 봅시다.<br />
For example, we can divide the years of schooling by the age.

In [ ]:
ind['escolari/age'] = ind['escolari'] / ind['age']

plt.figure(figsize = (10, 8))
sns.violinplot('Target', 'escolari/age', data = ind);

We can also take our new variable, inst, and divide this by the age. The final variable we'll name tech: this represents the combination of tablet and mobile phones.

In [ ]:
ind['inst/age'] = ind['inst'] / ind['age']
ind['tech'] = ind['v18q'] + ind['mobilephone']
ind['tech'].describe()

## Feature Engineering through Aggregations<br />
In order to incorporate the individual data into the household data, we need to aggregate it for each household. <br />
=> 개별 데이터를 가구별 데이터로 변환하기 위해서 각각의 가구별로 병합을 할 필요가 있습니다.<br />
The simplest way to do this is to groupby the family id idhogar and then agg the data. <br />
=> 가장 간단한 방법으로는 가구별 식별자로 그룹후에 그룹함수를 사용하는 것입니다<br />
For the aggregations for ordered or continuous variables, we can use six, five of which are built in to pandas, and one of which we define ourselves range_. <br />
=> 순서 및 연속적인 변수에 대해서 그룹함수를 사용하는데 있어서 여섯가지가 있으며, 한가지는 직접 정의해서 사용할 것입니다.<br />
The boolean aggregations can be the same, but this will create many redundant columns which we will then need to drop. <br />
=> 논리형 변수도 결합하는 것이 같지만, 이는 우리가 나중에 버릴 수도 있는 중복된 컬럼들이 생기는 경우가 될 수 있습니다.<br />
For this case, we'll use the same aggregations and then go back and drop the redundant columns.<br />
=> 우리는 같은 결합정책을 사용하고 나중에 돌아가서 중복되는 컬럼들을 제거 할 것입니다.

In [ ]:
range_ = lambda x:x.max() - x.min()
range_.__name__ = 'range_'

ind_agg = ind.drop(columns='Target').groupby('idhogar').agg(['min','max','sum','count','std',range_])
ind_agg.head()

In [ ]:
new_cols = []

for col in ind_agg.columns.levels[0]:
    for stat in ind_agg.columns.levels[1]:
        new_cols.append(f'{col}-{stat}')
        
ind_agg.columns = new_cols
ind_agg.head()

In [ ]:
ind_agg.shape

Feature Selection
As a first round of feature selection, we can remove one out of every pair of variables with a correlation greater than 0.95.

In [ ]:
corr_matrix = ind_agg.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

to_drop = [col for col in upper.columns if any(abs(upper[col]) > 0.95)]

print(f'There are {len(to_drop)} correlated columns to remove.')

We'll drop the columns and then merge with the heads data to create a final dataframe.

In [ ]:
ind_agg = ind_agg.drop(columns=to_drop)
ind_feats = list(ind_agg.columns)

final = heads.merge(ind_agg,on='idhogar',how='left')

print('Final features shape: ', final.shape)

In [ ]:
final.head()

Final Data Exploration
We'll do a little bit of exploration.

In [ ]:
corrs = final.corr()['Target']

In [ ]:
corrs.sort_values().head()

In [ ]:
corrs.sort_values().dropna().tail()

We can see some of the variables that we made are highly correlated with the Target. <br />
=> 타겟 라벨과 높은 관련성을 보이는 변수들을 찾아낼 수 있습니다.<br />
Whether these variables are actually useful will be determined in the modeling stage.<br />
=> 이 정보는 모델링 단계에서 유용할지 아닐지가 결정될 것입니다.<br />

In [ ]:
plot_categoricals('escolari-max', 'Target', final, annotate=False);

In [ ]:
plt.figure(figsize = (10, 6))
sns.violinplot(x = 'Target', y = 'escolari-max', data = final);
plt.title('Max Schooling by Target');

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x = 'Target', y = 'escolari-max', data = final);
plt.title('Max Schooling by Target');

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x = 'Target', y = 'meaneduc', data = final);
plt.xticks([0, 1, 2, 3], poverty_mapping.values())
plt.title('Average Schooling by Target');

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x = 'Target', y = 'overcrowding', data = final);
plt.xticks([0, 1, 2, 3], poverty_mapping.values())
plt.title('Overcrowding by Target');

One other feature that might be useful is the gender of the head of household. <br />
=> 다른 중요할 것으로 예상되는 특징은 가장의 성별입니다.<br />
Since we aggregated the data, we'll have to go back to the individual level data and find the gender for the head of household.<br />
=> 우리가 데이터를 합친 이후로, 이제 개별 데이터에 대해서 돌아보고 가장의 성별에 대해서 알아봅시다.

In [ ]:
head_gender = ind.loc[ind['parentesco1'] == 1, ['idhogar', 'female']]
final = final.merge(head_gender, on = 'idhogar', how = 'left').rename(columns = {'female': 'female-head'})

In [ ]:
final.groupby('female-head')['Target'].value_counts(normalize=True)

It looks like households where the head is female are slightly more likely to have a severe level of poverty.

In [ ]:
sns.violinplot(x = 'female-head', y = 'Target', data = final);
plt.title('Target by Female Head of Household');

We can also look at the difference in average education by whether or not the family has a female head of household.

In [ ]:
plt.figure(figsize = (8, 8))
sns.boxplot(x = 'Target', y = 'meaneduc', hue = 'female-head', data = final);
plt.title('Average Education by Target and Female Head of Household', size = 16);

It looks like at every value of the Target, households with female heads have higher levels of education. Yet, we saw that overall, households with female heads are more likely to have severe poverty.

In [ ]:
final.groupby('female-head')['meaneduc'].agg(['mean', 'count'])

Overall, the average education of households with female heads is slightly higher than those with male heads. <br />
=> 전반적으로 가장이 남자인 경우보다 여자인 경우가 더 교육기간이 긴 것을 확인할 수 있다.<br />
I'm not too sure what to make of this, but it seems right to me.<br />
=> 왜 이렇게 되었는지는 모르겠지만 이러한 통찰을 얻을 수 있었습니다.

## Machine Learning Modeling<br />
Once feature engineering/construction is done, we can get started with the machine learning! <br />
=> feature engineering/construction이 끝나고 난후에 우리는 머신러닝 모델을 만들 시간입니다.<br />
All of our data (both training and testing) is aggregated for each household and so can be directly used in a model. <br />
=> 우리의 모든 데이터는 각각의 가구별로 이미 나누어졌습니다. 이제 이를 바로 모델에 사용하면 될 것 같습니다.<br />
To first show the process of modeling, we'll use the capable Random Forest Classifier in Scikit-Learn.<br />
=> 모델링의 첫번째 단계로 RandonForestClassifier를 사용할 것입니다.<br />
This probably won't get us to the top of the leaderboard, but it will allow us to establish a baseline.<br />
=> 이는 점수판에 최상위에 오르게는 하지 않지만 적절한 초석을 다지기엔 좋습니다.<br />
Later we'll try several other models including the powerful Gradient Boosting Machine.<br />
=> 추후에 Gradient Boosting을 활용하여 더 강력한 모델을 만들겠습니다

To assess our model, we'll use 10-fold cross validation on the training data.<br />
=> 우리의 모델을 평가하기 위해서 10겹 교차검증을 사용하겠습니다<br />
This will essentially train and test the model 10 times using different splits of the training data. <br />
=> 이는 10번에 거쳐서 10겹에 나눠진 데이터 별로 훈련과 테스트를 진행합니다.<br />
10-fold cross validation is an effective method for estimating the performance of a model on the test set. <br />
=> 10겹 교차검증은 모델의 성능을 측정하는데 효과적인 방법입니다.<br />
We want to look at the average performance in cross validation as well as the standard deviation to see how much scores change between the folds. <br />
=> 우리는 일반적인 방법을 거친것과 10겹 교차검증을 거친 것의 점수 또한 비교해 볼 것입니다.<br />
We use the F1 Macro measure to evaluate performance.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Custom scorer for cross validation
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

In [ ]:
train_labels = np.array(list(final.loc[final['Target'].notnull(),'Target'].astype(np.uint8)))

train_set = final[final['Target'].notnull()].drop(columns=['Id','idhogar','Target'])

test_set = final[final['Target'].isnull()].drop(columns=['Id','idhogar','Target'])

submission_base = test[['Id', 'idhogar']].copy()

In [ ]:
train_set.head()

Because we are going to be comparing different models, we want to scale the features (limit the range of each column to between 0 and 1).<br />
=> 우리는 다른 모델을 비교할 것이기 때문에 우리의 특징들을 scaling할 필요가 있어보입니다.<br />
For many ensemble models this is not necessary, but when we use models that depend on a distance metric, such as KNearest Neighbors or the Support Vector Machine, feature scaling is an absolute necessity. <br />
=> 대부분의 앙상블 모델에는 불필요하지만, 거리와 관련된 척도인 K 최근접 이웃이나 서포트 벡터머신 과 같은 머신러닝 모델을 이용한다면 이런 스케일링은 필수적입니다.<br />
When comparing different models, it's always safest to scale the features. <br />
=> 다른 모델을 비교할 때 스케일링하는 것은 가장 안전한 방법중에 하나입니다.<br />
We also impute the missing values with the median of the feature.<br />
=> 그리고 우리는 각 특징의 중앙값을 손실치들에 넣어줄 것입니다.

**For imputing missing values and scaling the features in one step, we can make a pipeline.** <br />
**=> 파이프라인을 이용하면 손실치를 넣거나 스케일링을 하는데 간단하게 처리할 수있습니다.**<br />
This will be fit on the training data and used to transform the training and testing data.

In [ ]:
features = list(train_set.columns)

pipeline = Pipeline([('imputer',Imputer(strategy='median')),('scaler',MinMaxScaler())])

train_set = pipeline.fit_transform(train_set)
test_set = pipeline.transform(test_set)

The data has no missing values and is scaled between zero and one. This means it can be directly used in any Scikit-Learn model.

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=10, n_jobs = -1)
# 10 fold cross validation
cv_score = cross_val_score(model, train_set, train_labels, cv = 10, scoring = scorer)

print(f'10 Fold Cross Validation F1 Score = {round(cv_score.mean(), 4)} with std = {round(cv_score.std(), 4)}')

That score is not great, but it will serve as a baseline and leaves us plenty of room to improve!

## Feature Importances<br />
With a tree-based model, we can look at the feature importances which show a relative ranking of the usefulness of features in the model. <br />
=> 트리 모델로 우리는 모델에서 특징의 필요없는 정도를 나타내는 상대적 랭킹을 볼 수 있습니다.<br />
These represent the sum of the reduction in impurity at nodes that used the variable for splitting, but we don't have to pay much attention to the absolute value. Instead we'll focus on relative scores.<br />
=> 이는 변수를 분기하는데 사용되는 각각의 노드에 불 순수성에서 reduction의 합을 나타내며, 이들의 실제값에는 많은 관심을 두지 않습니다. 대신에 우리는 상대적인 점수에 초점을 맞출 것입니다.

If we want to view the feature importances, we'll have to train a model on the whole training set. Cross validation does not return the feature importances.<br />
만약 feature_importances 의 수치를 보고 싶다면, 모델을 전체 트레이닝 셋에 훈련시키고 해라.

In [ ]:
model.fit(train_set,train_labels)

indices = np.argsort(model.feature_importances_)[::-1]
importances = model.feature_importances_

for i in range(len(features)):
    print("%d) feature importances of %s %f"%(i,features[indices[i]],importances[indices[i]]))

In [ ]:
# def plot_feature_importances(df,feature_label,importance_label,n=10,ascending=False):
    
#     plt.style.use('fivethirtyeight')
    
#     df = df.sort_values(by=importance_label,ascending=ascending).reset_index()
#     df.loc[list(range(n)),][importance_label].plot(kind='bar')
#     plt.xticks(list(range(n)),df.loc[list(range(n)),][feature_label])

In [ ]:
def plot_feature_importances(df, n = 10, threshold = None):
    """Plots n most important features. Also plots the cumulative importance if
    threshold is specified and prints the number of features needed to reach threshold cumulative importance.
    Intended for use with any tree-based feature importances. 
    
    Args:
        df (dataframe): Dataframe of feature importances. Columns must be "feature" and "importance".
    
        n (int): Number of most important features to plot. Default is 15.
    
        threshold (float): Threshold for cumulative importance plot. If not provided, no plot is made. Default is None.
        
    Returns:
        df (dataframe): Dataframe ordered by feature importances with a normalized column (sums to 1) 
                        and a cumulative importance column
    
    Note:
    
        * Normalization in this case means sums to 1. 
        * Cumulative importance is calculated by summing features from most to least important
        * A threshold of 0.9 will show the most important features needed to reach 90% of cumulative importance
    
    """
    plt.style.use('fivethirtyeight')
    
    # Sort features with most important at the head
    df = df.sort_values('importance', ascending = False).reset_index(drop = True)
    
    # Normalize the feature importances to add up to one and calculate cumulative importance
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])
    
    plt.rcParams['font.size'] = 12
    
    # Bar plot of n most important features
    df.loc[:n, :].plot.barh(y = 'importance_normalized', 
                            x = 'feature', color = 'darkgreen', 
                            edgecolor = 'k', figsize = (12, 8),
                            legend = False, linewidth = 2)

    plt.xlabel('Normalized Importance', size = 18); plt.ylabel(''); 
    plt.title(f'{n} Most Important Features', size = 18)
    plt.gca().invert_yaxis()
    
    
    if threshold:
        # Cumulative importance plot
        plt.figure(figsize = (8, 6))
        plt.plot(list(range(len(df))), df['cumulative_importance'], 'b-')
        plt.xlabel('Number of Features', size = 16); plt.ylabel('Cumulative Importance', size = 16); 
        plt.title('Cumulative Feature Importance', size = 18);
        
        # Number of features needed for threshold cumulative importance
        # This is the index (will need to add 1 for the actual number)
        importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
        
        # Add vertical line to plot
        plt.vlines(importance_index + 1, ymin = 0, ymax = 1.05, linestyles = '--', colors = 'red')
        plt.show();
        
        print('{} features required for {:.0f}% of cumulative importance.'.format(importance_index + 1, 
                                                                                  100 * threshold))
    
    return df

In [ ]:
temp_df = pd.DataFrame({'feature':features,'importance':importances})

In [ ]:
norm_fi = plot_feature_importances(temp_df, threshold=0.95)

Education reigns supreme! The most important variable is the average amount of education in the household, followed by the maximum education of anyone in the household. I have a suspicion these variables are highly correlated (collinear) which means we may want to remove one of them from the data. The other most important features are a combination of variables we created and variables that were already present in the data. <br />
=>  교육은 정말 중요해보여요! 가장 중요한 변수는 각 가정에서 교육의 평균양이었스비다. 그리고 나는 이러한 변수들이 상당히 연관관계가 클 것 같습니다. 그래서 우리는 그들을 지워주려고 합니다. 다른 중요한 특징흔 우리가 만들었넌 변수들의 조합이고 그것들은 이미 데이터 안에 존재합니다 

It's interesting that we only need 106 of the ~180 features to account for 90% of the importance. This tells us that we may be able to remove some of the features. However, feature importances don't tell us which direction of the feature is important (for example, we can't use these to tell whether more or less education leads to more severe poverty) they only tell us which features the model considered relevant.<br />
=>106에서 180개 정도의 피쳐만 있어도 90%의 importance를 달성한 것을 알 수 있습니다. 이는 우리가 몇몇 feature들을 지워줘야 한다는 이야기 입니다. 그러나 feature_importances는 특징의 어느 방향이 중요한지 말해주지 않습니다. 그들은 단지 어느 특징이 모델과 관련이 있었는지만 알려줍니다. (어느 컬럼이 부의 수준과 관련이 있다는 것은 이야기 해주지 않습니다.)

In [ ]:
def kde_target(df, variable):
    """Plots the distribution of `variable` in `df` colored by the `Target` column"""
    
    colors = {1: 'red', 2: 'orange', 3: 'blue', 4: 'green'}

    plt.figure(figsize = (12, 8))
    
    df = df[df['Target'].notnull()]
    
    for level in df['Target'].unique():
        subset = df[df['Target'] == level].copy()
        sns.kdeplot(subset[variable].dropna(), 
                    label = f'Poverty Level: {level}', 
                    color = colors[int(subset['Target'].unique())])

    plt.xlabel(variable); plt.ylabel('Density');
    plt.title('{} Distribution'.format(variable.capitalize()));

In [ ]:
kde_target(final, 'meaneduc')

In [ ]:
kde_target(final, 'escolari/age-range_')

**Model Selection** <br />
Now that we have a good set of features, it's time to get into the modeling. <br />
=> 이제 우리는 좋은 변수들의 집합을 가지고 있습니다. 이제 모델링을 해볼 차례입니다.
We already tried one basic model, the Random Forest Classifier which delivered a best macro F1 of 0.35. <br />
=> 우리는 이미 RandomForestClassifier로 0.35점의 성과를 냈습니다. <br />
However, in machine learning, there is no way to know ahead of time which model will work best for a given dataset. <br />
=> 그러나 머신러닝에서 어느 모델이 주어진 데이터셋에서 최고로 작동을 자랄지 아는 방법이 없습니다. <br />
The following plot shows that there are some problems where even Gaussian Naive Bayes will outperform a gradient boosting machine. <br />
=> 다음의 그림은 가우시안 나이브 베이즈가 그라디언트 부스팅보다 좋은 성적을 낸적이 있는 문제도 있다는 것입니다.<br />
This is from an excellent paper by Randal Olson that discusses many points of machine learning

What this plot tells us is that we have to try out a number of different models to see which is optimal. <br />
=> 이 플롯이 말하고자 하는 바는 어느 것이 최적인지를 찾기위해서는 여러가지 다른 모델들을 테스트 해봐야 한다는 것입니다. <br />
Most people eventually settle on the gradient boosting machine and we will try that out, but for now we'll take a look at some of the other options. There are literally dozens (maybe hundreds) of multi-class machine <br />
=>많은 사람들이 끝내 그라디언트 부스팅에 머물지만, 우리는 다른 선택지들을 좀 살펴볼 필요가 있다. 여기에 굉장히 많은 선택지들이 있다.<br />
learning models if we look at the Scikit-Learn documentation. We don't have to try them all, but we should sample from the options. <br />
=> 다는 해보지 못하겠지만 일부라도 실험해보도록 하자 <br /> 

What we want to do is write a function that can evaluate a model <br />. 
=> 우리는 각 모델을 을 평가할 수 있는 함수를 짜야한다. <br />
This will be pretty simple since we already wrote most of the code. 
=> 이는 간단할 것이다.
In addition to the Random Forest Classifier, we'll try eight other Scikit-Learn models. <br />
Luckily, this dataset is relatively small and we can rapidly iterate through the models. <br />
=> 운좋게도 우리의 모델은 작기 때문에 상대적으로 빠르게 될 것이다 <br />
We will make a dataframe to hold the results and the function will add a row to the dataframe for each model. <br />
=> 결과를 담을 데이터프레임을 만들고 각모델에 대해서 결과를 데이터프레임에 행에 추가해 줄 것이다.<br />

In [ ]:
# Model imports
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import warnings 
from sklearn.exceptions import ConvergenceWarning

# Filter out warnings from models
warnings.filterwarnings('ignore', category = ConvergenceWarning)
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = UserWarning)

# Dataframe to hold results
model_results = pd.DataFrame(columns = ['model', 'cv_mean', 'cv_std'])

def cv_model(train, train_labels, model, name, model_results=None):
    """Perform 10 fold cross validation of a model"""
    
    cv_scores = cross_val_score(model, train, train_labels, cv = 10, scoring=scorer, n_jobs = -1)
    print(f'10 Fold CV Score: {round(cv_scores.mean(), 5)} with std: {round(cv_scores.std(), 5)}')
    
    if model_results is not None:
        model_results = model_results.append(pd.DataFrame({'model': name, 
                                                           'cv_mean': cv_scores.mean(), 
                                                            'cv_std': cv_scores.std()},
                                                           index = [0]),
                                             ignore_index = True)

        return model_results

In [ ]:
model_results = cv_model(train_set,train_labels,LinearSVC(),'LSVC',model_results)

That's one model to cross off the list (although we didn't perform hyperparameter tuning so the actual performance could possibly be improved). <br />

In [ ]:
model_results = cv_model(train_set,train_labels,GaussianNB(),'GNB',model_results)

That performance is very poor. I don't think we need to revisit the Gaussian Naive Bayes method (although there are problems on which it can outperform the Gradient Boosting Machine). <br />
=> 가우시안 나이브 베이즈 모델의 성능은 굉장히 형편없었다.

In [ ]:
model_results = cv_model(train_set,train_labels,MLPClassifier(hidden_layer_sizes=(32,64,128,64,32)),'MLP',model_results)

The multi-layer perceptron (a deep neural network) has decent performance. This might be an option if we are able to hyperparameter tune the network. However, the limited amount of data could be an issue with a neural network as these generally require hundreds of thousands of examples to learn effectively. <br />
=> 다층 퍼세브론은 굉장한 성능을 보여주었습니다. 우리가 튜닝을 좀한다면 더 좋아질 것 같습니다. 그러나 신경망에서 작은 양의 데이터는 문제가 될 수 있습니다.

In [ ]:
model_results = cv_model(train_set, train_labels, 
                          LinearDiscriminantAnalysis(), 
                          'LDA', model_results)

If you run LinearDiscriminantAnalysis without filtering out the UserWarnings, you get many messages saying "Variables are collinear." This might give us a hint that we want to remove some collinear features! We might want to try this model again after removing the collinear variables because the score is comparable to the random forest. <br />
=> 해당 모델을 경고창을 없애지 않고 한다면 위와 같은 경고메시지를 만날 수 있는데 이는, 유사한 성격을 가진 컬럼이 많아서 다중 공선성의 문제가 생길 수 있기 때문입니다. 하지만 우리가 제일 처음에 가이드 라인으로 사용했던 랜덤포레스트와 비슷한 점수를 내기 때문에 공선성 문제를 해결하고 이 모델을 다시 시험해 볼 것입니다.

In [ ]:
model_results = cv_model(train_set, train_labels, RidgeClassifierCV(), 'RIDGE', model_results)

The linear model (with ridge regularization) does surprisingly well. This might indicate that a simple model can go a long way in this problem (although we'll probably end up using a more powerful method). <br />
=> 선형모델은 생각보다 잘 되었습니다. 


In [ ]:
for n in [5, 10, 20]:
    print(f'\nKNN with {n} neighbors\n')
    model_results = cv_model(train_set, train_labels, 
                             KNeighborsClassifier(n_neighbors = n),
                             f'knn-{n}', model_results)

As one more attempt, we'll consider the ExtraTreesClassifier, a variant on the random forest using ensembles of decision trees as well. <br />
=> 마지막으로는 결정트리에 앙상블을 이용한 랜덤포레스트의 변형인 ExtraTreeClassifier를 이용해보겠습니다.


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model_results = cv_model(train_set, train_labels, 
                         ExtraTreesClassifier(n_estimators = 100, random_state = 10),
                         'EXT', model_results)

## Comparing Model Performance
With the modeling results in a dataframe, we can plot them to see which model does the best.

In [ ]:
model_results = cv_model(train_set, train_labels, 
                         RandomForestClassifier(n_estimators = 100, random_state = 10),
                         'RF', model_results)

In [ ]:
models = model_results['model']

In [ ]:
model_results_ = model_results.set_index('model')
model_results_['cv_mean'].plot(kind='bar',color='orange',figsize=(8,6),edgecolor='k',linewidth=2, yerr=list(model_results['cv_std']))

The most likely candidate seems to be the Random Forest because it does best right out of the box. <br />
=> 많은 사람들은 랜덤포레스트가 최고의 방법이라고 볼 수 있다. <br />
While we didn't tune any of the hyperparameters so the comparison between models is not perfect, these results reflect those of many other Kaggle competitiors finding that tree-based ensemble methods (including the Gradient Boosting Machine) perform very well on structured datasets. <br />
=> 그러나 우리는 아무런 파라미터 튜닝을 하지 않았기 때문에 이 모델들은 완벽하지 않다, 이 결과는 왜 많은 캐글의 이용자들이 트리기반의 아앙블 메소드를 쓰는지 보여준다 <br />
Hyperparameter performance does improve the performance of machine learning models, but we don't have time to try all possible combinations of settings for all models.  <br />
=> 하이퍼 파라미터 튜닝은 머신런이 모델의 성능을 향상시켜주지만, 모든 모델의 파라미터 튜닝을 하고 있기엔 시간이 없다 <br />
The graph below (from the paper by Randal Olson) shows the effect of hyperparameter tuning versus the default values in Scikit-Learn. 

In most cases the accuracy gain is less than 10% so the worst model is probably not suddenly going to become the best model through tuning. <br />
=> 대부분의 경우에 정확도를 많아도 10퍼센트까지만 올릴 수 있다. 

For now we'll say the random forest does the best. Later we'll look at using the Gradient Boosting Machine, although not implemented in Scikit-Learn. <br />
=> 지금은 랜덤포레스트가 최고라고 말할 수 있지만, 나중에 우리가 사용하게될 그라디언트 부스팅 모델에 대해서 알아볼 것이다. <br />
Instead we'll be using the more powerful LightGBM version. Now, let's turn to making a submission using the random forest. <br />
=> 우리는 더 강력한 LightGBM모델을 사용할 것이다.

## Making a Submission
In order to make a submission, we need the test data. Fortunately, we have the test data formatted in exactly the same manner as the train data. <br />
=> 제출을 위해, 우리는 테스트 데이터가 필요하다. 운좋게도 우리는 테스트 데이터를 훈련데이터를 전처리한 것과 동일하게 처리해 두었다.

The format of a testing submission is shown below. Although we are making predictions for each household, we actually need one row per individual (identified by the Id) but only the prediction for the head of household is scored. <br />
=>  테스트 셋의 결과 제출은 아래와 같다. 비록 우리가 모든 가구들에 대해 결과를 예측했더라도 우리는 개인 row당 하나의 타겟 값을 원한다. 그러나 가장의 것만 점수에 합산 될 것이다.

Id,Target <br />
ID_2f6873615,1 <br />
ID_1c78846d2,2 <br />
ID_e5442cf6a,3 <br />
ID_a8db26a79,4 <br />
ID_a62966799,4 <br />

The submission_base will have all the individuals in the test set since we have to have a "prediction" for each individual while the test_ids will only contain the idhogar from the heads of households. <br />
=> submissin_base는 모든 개개인에 대한 정보를 다 가지고 있다. 그런데 우리가 기잔 test_id는 가구별로 예측이 되어있다 <br />
When predicting, we only predict for each household and then we merge the predictions dataframe with all of the individuals on the household id (idhogar). <br />
=> 예측시에, 우리는 가구 단위로 예측을 했고, 우리는 두 개의 데이터프레임을 left outer join을 통해 모든 가구원들에게 가구별 타겟 값을 할당할 것이다. <br />
This will set the Target to the same value for everyone in a household. For the test households without a head of household, we can just set these predictions to 4 since they will not be scored. <br />
=> 이것은 모든 구성원들에게 값이 돌아가지만, 가장이 없는 가구의 경우에는 예측이 되지 않기 때문에 4로 배정한다.

In [ ]:
test_ids = list(final.loc[final['Target'].isnull(),'idhogar'])

The function below takes in a model, a training set, the training labels, and a testing set and performs the following operations: <br />

Trains the model on the training data using fit<br />
Makes predictions on the test data using predict<br />
Creates a submission dataframe that can be saved and uploaded to the competition<br />

In [ ]:
def submit(model,train,train_labels,test,test_ids):
    
    model.fit(train,train_labels)
    predictions = model.predict(test)
    predictions = pd.DataFrame({'idhogar':test_ids,'Target':predictions})
    
    submission = submission_base.merge(predictions,on='idhogar',how='left').drop(columns=['idhogar'])
    
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
    
    return submission

In [ ]:
rf_submission = submit(RandomForestClassifier(n_estimators = 100, 
                                              random_state=10, n_jobs = -1), 
                         train_set, train_labels, test_set, test_ids)

rf_submission.to_csv('rf_submission.csv', index = False)

## Feature Selection 
One potential method for improving model performance is feature selection. This is the process where we try to keep only the most useful features for our model. "Most useful" can mean many different things, and there are numerous heuristics for selecting the most important features. For feature selection in this notebook, we'll first remove any columns with greater than 0.95 correlation (we already did some of this during feature engineering) and then we'll apply recursive feature elimination with the Scikit-Learn library. <br />
모델의 성능을 평가하기 위한 하나의 방법은 feature_selection이다. 이는 우리의 모델에서 유의미한 컬럼들만 남기기 위해서 진행하는 것이다. 가장 유용하다는 것은 많은 다른 것들을 의미하고, 이런 가장 중요한 컬럼들 찾는데 많은 경험적 지식들이 있다. 이 커널에서는 우리는 0.95이상의 상관관께 이상을 가지는 임의의 컬럼들을 지우고, **Recursive feature elimination** 기법을 이용할 것이다


First up are the correlations. 0.95 is an arbitrary threshold - feel free to change the values and see how the performance changes! <br />
일단은 0.95를 임계점으로 하여 컬럼들을 찾아내자


In [ ]:
train_set = pd.DataFrame(train_set,columns=features)

In [ ]:
corr_matrix = train_set.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

to_drop = [column for column in upper.columns if any(abs(upper[column]) >0.95)]

to_drop

In [ ]:
print(train_set.shape)

In [ ]:
train_set = train_set.drop(to_drop,axis=1)
print(train_set.shape)

In [ ]:
test_set = pd.DataFrame(test_set,columns=features)
print(test_set.shape)
train_set, test_set = train_set.align(test_set,axis=1,join='inner')
features = list(train_set.columns)
print(test_set.shape)

## Recursive Feature Elimination with Random Forest
The RFECV in Sklearn stands for Recursive Feature Elimination with Cross Validation. The selector operates using a model with feature importances in an iterative manner. At each iteration, it removes either a fraction of features or a set number of features. The iterations continue until the cross validation score no longer improves. <br />
=> RFECV는 교차검증을 이용한 반복적 특징 제거법입니다. 선택자는 반복적인 방법을 통해서 모델의 feature_importance를 진행합니다. 각각의 반복에서, RFECV는 특징은 부분이나 특징의 개수를 점점 줄여갑니다. 이 반복은 더이상 교차검증 점수가 향상되지 않을 때 까지 진행합니다.

To create the selector object, we pass in the the model, the number of features to remove at each iteration, the cross validation folds, our custom scorer, and any other parameters to guide the selection. <br />
선택자 객체를 생성하기 위해, 우리는 모델을 넘기고, 각각의 반복시행마다 없앨 컬럼들의 개수, 교차검증의 겹수, 우리의 custom scorer 그리고 선택자가 참고할 수 있는 다른 파라미터들을 넘겨줍니다.

In [ ]:
from sklearn.feature_selection import RFECV

estimator = RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)

selector = RFECV(estimator,step=1,cv=5,scoring=scorer,n_jobs=-1)

Then we fit the selector on the training data as with any other sklearn model. This will continue the feature selection until the cross validation scores no longer improve. <br />
=> 다음으로는 선택자를 훈련세트를 이용하여 학습시킵니다. 이는 교차검증점수가 더 이상 낮아질 때가 없을 때 까지 계속해서 진행 할 것입니다.

In [ ]:
selector.fit(train_set,train_labels)

We can investigate the object to see the training scores for each iteration. The following code will plot the validation scores versus the number of features for the training. <br />
=> 우리는 각각의 반복마다 훈련 점수를 볼 수 있습니다. 다음의 코드는 교차검증 점수와 컬럼의 개수를 그림화 한 것입니다.

In [ ]:
plt.plot(selector.grid_scores_)

plt.xlabel('Number of Features')
plt.ylabel('Macro F1 Score')
plt.title('Feature Selection Scores')
selector.n_features_

We can see that the score improves as we add features up until 96 features. According to the selector, this is the optimal number of features. <br />
우리는 96개의 feature를 넣을 때가지만 해도 점수의 향상을 알 수 있었습니다. selector에 따르면 최적의 피쳐의 수는 정해졌습니다.

The rankings of each feature can be found by inspecting the trained object. These represent essentially the importance of features averaged over the iterations. Features can share the same ranking, and only features with a rank of 1 are retained. <br />
각각의 컬럼들의 랭킹은 훈련된 선택자 객체를 통해 알 수 있습니다. 이들은 반복을 거듭함에 따라 평균된 중요도를 나타냅니다. 특징들은 같은 랭킹을 공유할수 있습니다. rank가 1인 컬럼들만 사용될 것입니다.

In [ ]:
rankings = pd.DataFrame({'feature':list(train_set.columns),'rank':list(selector.ranking_)}).sort_values(by='rank')
rankings.head(10)

Finally, we select the features and then evaluate in cross validation. <br />
=> 마지막으로 우리들의 교차검증 결과를 확인해 봅시다.

In [ ]:
train_selected = selector.transform(train_set)
test_selected = selector.transform(test_set)

In [ ]:
train_selected

In [ ]:
# np.where(selector.ranking_==1)

In [ ]:
selected_features = train_set.columns[np.where(selector.ranking_==1)]
train_selected = pd.DataFrame(train_selected,columns=selected_features)
test_selected = pd.DataFrame(test_selected,columns=selected_features)

In [ ]:
train_selected.shape

In [ ]:
model_results = cv_model(train_selected,train_labels,model,'RF-SEL',model_results)

In [ ]:
model_results.set_index('model', inplace = True)
model_results_ = model_results

model_results_['cv_mean'].plot.bar(color = 'orange', figsize = (8, 6),
                                  yerr = list(model_results['cv_std']),
                                 edgecolor = 'k', linewidth = 2)
plt.title('Model F1 Score Results');
plt.ylabel('Mean F1 Score (with error bar)');
model_results.reset_index(inplace = True)

# Upgrading Our Model: Gradient Boosting Machine
After using the Random Forest and getting decent scores, it's time to step up and use the gradient boosting machine. If you spend any time on Kaggle, you'll notice that the Gradient Boosting Machine (GBM) wins a high percentage of competitions where the data is structured (in tables) and the datasets are not that large (less than a million observations). <br />
=> 우리가 랜덤포레스트를 통해 좋은 점수를 받은 이후에, 그라디언트 부스팅모델을 이용할 차례입니다. 만약 독자가 캐글에 익숙한 사용자라면 GBM모델이 많은 데이터가 정형화되어있고 데이터셋이 크지 않은 컴피티션에서 승리한 것을 보았을 것입니다.<br />

I won't go too much into the details here, but instead will focus on the implementation. We'll use the GBM in LightGBM, although there are also options in Scikit-Learn, XGBOOST, and CatBoost. The first set of hyperparameters we'll use were based on those I've found have worked well for other problems. <br />
=> 많은 자세한 것들을 언급하진 않겠지만 실행에 초점을 맞춰봅시다. 우리는 LightGBM을 사용할 것이지만 다른 여러 선택지가 사이킷런,XGBOOST,CatBoost가 있습니다. 첫번 째 하이퍼 파라미터의 세트는 우리가 위에서 찾아왔던 것들을 기반으로 세팅하겠습니다.<br />

## Choosing Number of Estimators with Early Stopping
To choose the number of estimators (the number of decision trees in the ensemble, called n_estimators or num_boost_rounds), we'll use early stopping with 5-fold cross validation. This will keep adding estimators until the performance as measured by the Macro F1 Score has not increased for 100 training rounds. To use this metric, we'll have to define a custom metric. <br />
=>반복시행의 수를 선택하기 위해 우리는 5겹 교차검증을 이용할 것입니다. 이것은 우리가 이용하는 척도 Macro F1 수치가 100번의 훈련 반복시행에 있어서 증가하지 않을 때 까지 반복시행을 추가합니다. 이 수치를 이용하기 위해 custom metric을 이용합시다.<br />

In [ ]:
def macro_f1_score(labels, predictions):
    # Reshape the predictions as needed
    # 행으로만 이루어진 결과치를 4열씩 차례대로 배치하고 그 4열에서 max를 각각 추출
    predictions = predictions.reshape(len(np.unique(labels)), -1 ).argmax(axis = 0)
    
    metric_value = f1_score(labels, predictions, average = 'macro')
    
    # Return is name, value, is_higher_better
    return 'macro_f1', metric_value, True

## Light Gradient Boosting Machine Implementation
The function below implements training the gradient boosting machine with Stratified Kfold cross validation and early stopping to prevent overfitting to the training data (although this can still occur). The function performs training with cross validation and records the predictions in probability for each fold. To see how this works, we can return the predictions from each fold and then we'll return a submission to upload to the competition. <br />
=> 아래의 함수는 K겹 교차검증을 이용한 그라디언트 부스팅 모델을 학습시키는 함수이며, early stopping은 훈련데이터가 과적합되는 것을 막는 파라미터입니다. 이 함수는 교차검증을 통해 훈련시키고 각 계층마다의 예측에 대한 확률을 기록합니다. 이것이 어떻게 작동하는지를 알기 위해서 각 폴드별 예측치를 반환하는 prediction을 반환하고 컴피티션의 업로드를 위해서 submission 또한 업로드 할 것 입니다.<br />

Choosing hyperparameters for the Gradient Boosting Machine can be tough and generally is done through model optimization. In this notebook, we'll use a set of hyperparameters that I've found work well on previous problems (although they will not necessarily translate to this competition). <br />
=> 그라디언트 부스팅에서 하이퍼파라미터를 고르는 것은 어려운일입니다. 그리고 이는 모델 최적화를 통해 이루어질 것입니다. 여기서는 일련의 하이퍼 파라미터들을 세팅할 것입니다.<br />

We set the n_estimators to 10000 but we won't actually reach this number because we are using early stopping which will quit training estimators when the cross validation metric does not improve for early_stopping_rounds. There's a lot going on in this function, and read through it carefully to make sure you have it all! I've tried to make the comments and code straightforward. (The display is used to show custom information during training in combination with %%capture so we don't have to see all the LightGBM information during training). <br />
=> 우리는 n_estimators를 10000으로 세팅하였지만, 우리는 교차검증 점수가 더 이상 오르지 않는 순간을 캐치하고 이를 중단 시키는 early_stopping을 설정하기 때문에 만 번의 반복 시행까지는 가지 않을 것입니다. 많은 것들이 이 함수에서 이루어집니다. 이를 모두 읽고 여러분의 것으로 만드세요. 코드에 주석들을 달아 놨습니다.<br />

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgb

# feature_names = features

# params = {
#     'boosting_type':'dart',
#     'colsample_bytree':0.88,
#     'learning_rate':0.028,
#     'min_child_samples':10,
#     'num_leaves':36, 'reg_alpha':0.76,
#     'reg_lambda':0.43,
#     'subsample_for_bin':40000,
#     'subsample':0.54,
#     'class_weight':'balanced'
# }

# model = lgb.LGBMClassifier(**params,objective='multiclass',n_jobs=-1,n_estimators=1000,random_state=10)

# SKFOLD = StratifiedKFold(n_splits=5,shuffle=True)

# predictions = pd.DataFrame()
# importances = np.zeros(len(feature_names))

# features = np.array(train_set)
# test_features = np.array(test_set)
# labels = np.array(train_labels).reshape((-1))

# valid_scores = []

# for i,(train_indices,valid_indices) in enumerate(SKFOLD.split(features,labels)):
    
#     fold_predictions = pd.DataFrame()
    
#     X_train,X_valid = features[train_indices],features[valid_indices]
#     y_train,y_valid = labels[train_indices],labels[valid_indices]
    
#     model.fit(X_train,y_train,early_stopping_rounds=100,eval_metric=macro_f1_score,eval_set=[(X_train,y_train),(X_valid,y_valid)],eval_names=['train','valid'],verbose=200)
    
#     valid_scores.append(model.best_score_['valid']['macro_f1'])
    
#     fold_probabilities = model.predict_proba(test_features)
    
#     for j in range(4):
#         fold_predictions[(j+1)] = fold_probabilities[:,j]
        
#     fold_predictions['idhogar'] = test_ids
#     fold_predictions['fold'] = (i+1)
    
#     predictions = predictions.append(fold_predictions)
    
#     importances += model.feature_importances_ / 5
    
#     display(f'Fold {i + 1}, Validation Score: {round(valid_scores[i], 5)}, Estimators Trained: {model.best_iteration_}')
    
#     predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
#     predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
    
#     submission = submission_base.merge(predictions[['idhogar','Target']],on='idhogar',how='left').drop(columns=['idhogar'])
#     submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
#     break


In [ ]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from IPython.display import display

def model_gbm(features, labels, test_features, test_ids, 
              nfolds = 5, return_preds = False, hyp = None):
    """Model using the GBM and cross validation.
       Trains with early stopping on each fold.
       Hyperparameters probably need to be tuned."""
    
    feature_names = list(features.columns)

    # Option for user specified hyperparameters
    if hyp is not None:
        # Using early stopping so do not need number of esimators
        if 'n_estimators' in hyp:
            del hyp['n_estimators']
        params = hyp
    
    else:
        # Model hyperparameters
        params = {'boosting_type': 'dart', 
                  'colsample_bytree': 0.88, 
                  'learning_rate': 0.028, 
                   'min_child_samples': 10, 
                   'num_leaves': 36, 'reg_alpha': 0.76, 
                   'reg_lambda': 0.43, 
                   'subsample_for_bin': 40000, 
                   'subsample': 0.54, 
                   'class_weight': 'balanced'}
    
    # Build the model
    model = lgb.LGBMClassifier(**params, objective = 'multiclass', 
                               n_jobs = -1, n_estimators = 10000,
                               random_state = 10)
    
    # Using stratified kfold cross validation
    strkfold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    
    # Hold all the predictions from each fold
    predictions = pd.DataFrame()
    importances = np.zeros(len(feature_names))
    
    # Convert to arrays for indexing
    features = np.array(features)
    test_features = np.array(test_features)
    labels = np.array(labels).reshape((-1 ))
    
    valid_scores = []
    
    # Iterate through the folds
    for i, (train_indices, valid_indices) in enumerate(strkfold.split(features, labels)):
        
        # Dataframe for fold predictions
        fold_predictions = pd.DataFrame()
        
        # Training and validation data
        X_train = features[train_indices]
        X_valid = features[valid_indices]
        y_train = labels[train_indices]
        y_valid = labels[valid_indices]
        
        # Train with early stopping
        model.fit(X_train, y_train, early_stopping_rounds = 100, 
                  eval_metric = macro_f1_score,
                  eval_set = [(X_train, y_train), (X_valid, y_valid)],
                  eval_names = ['train', 'valid'],
                  verbose = 200)
        
        # Record the validation fold score
        valid_scores.append(model.best_score_['valid']['macro_f1'])
        
        # Make predictions from the fold as probabilities
        fold_probabilitites = model.predict_proba(test_features)
        
        # Record each prediction for each class as a separate column
        for j in range(4):
            fold_predictions[(j + 1)] = fold_probabilitites[:, j]
            
        # Add needed information for predictions 
        fold_predictions['idhogar'] = test_ids
        fold_predictions['fold'] = (i+1)
        
        # Add the predictions as new rows to the existing predictions
        predictions = predictions.append(fold_predictions)
        
        # Feature importances
        importances += model.feature_importances_ / nfolds   
        
        # Display fold information
        display(f'Fold {i + 1}, Validation Score: {round(valid_scores[i], 5)}, Estimators Trained: {model.best_iteration_}')

    # Feature importances dataframe
    feature_importances = pd.DataFrame({'feature': feature_names,
                                        'importance': importances})
    
    valid_scores = np.array(valid_scores)
    display(f'{nfolds} cross validation score: {round(valid_scores.mean(), 5)} with std: {round(valid_scores.std(), 5)}.')
    
    # If we want to examine predictions don't average over folds
    if return_preds:
        predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
        predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
        return predictions, feature_importances
    
    # Average the predictions over folds
    predictions = predictions.groupby('idhogar', as_index = False).mean()
    
    # Find the class and associated probability
    predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
    predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
    predictions = predictions.drop(columns = ['fold'])
    
    # Merge with the base to have one prediction for each individual
    submission = submission_base.merge(predictions[['idhogar', 'Target']], on = 'idhogar', how = 'left').drop(columns = ['idhogar'])
        
    # Fill in the individuals that do not have a head of household with 4 since these will not be scored
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
    
    # return the submission and feature importances along with validation scores
    return submission, feature_importances, valid_scores

## Cross Validation with Early Stopping Notes

> 참고. Variance와 Bias https://www.slideshare.net/freepsw/boosting-bagging-vs-boosting [boosting 기법의 이해]

Cross validation with early stopping is one of the most effective methods for preventing overfitting on the training set because it prevents us from continuing to add model complexity once it is clear that validation scores are not improving. Repeating this process across multiple folds helps to reduce the bias that comes from using a single fold. Early stopping also lets us train the model much quicker. Overall, early stopping with cross validation is the best method to select the number of estimators in the Gradient Boosting Machine and should be our default technique when we desig an implementation. <br />
=> early stopping을 Cross validation 하는 것은 훈련세트에서 과적합을 막는 가장 효과적인 방법입니다. 왜냐하면 이는 우리가 모델 복잡도를 지속적으로 추가하는 것을 막기 때문이며, validation 점수가 향상되지 않을때 이는 명확해집니다. 여러겹을 거쳐서 이 과정을 반복하는 것은 하나의 단일 계층으로 했을 때 발생할 수 있는 bias를 줄여준다(bias가 낮으면 올바른 예측치를 찾아간다는 뜻 상충되는 개념으로 variance가 있음 위의 참고 url에 두 용어에 대한 개념이 있음). early stopping은 또한 우리가 훈련 모델 학습을 더 빠르게 해준다. 전반적으로 교차검증을 이용한 early stopping은 그라디언트 부스팅 모델에서 반복시행의 수를 선택하는 최고의 방법이며 실행을 하는데 있어서 가장 기초적인 지식이며 기술이다.  <br />

In [ ]:
predictions, gbm_fi = model_gbm(train_set,train_labels,test_set,test_ids,return_preds=True)

The power of the Gradient Boosting Machine can be seen here! The cross validation score blows away anything we've done previously. <br />
=> 그라디언트 부스팅 모델의 힘은 여기서 발휘된다. <br />

Let's take a look at the predictions to understand what is going on with the predictions in each fold. <br />
=> 각각의 계층별 진행된 예측이 어떻게 되었는지 이해를 위해 prediction 변수를 살펴보자<br />

In [ ]:
predictions.head()

For each fold, the 1, 2, 3, 4 columns represent the probability for each Target. The Target is the maximum of these with the confidence the probability. We have the predictions for all 5 folds, so we can plot the confidence in each Target for the different folds. <br />
각각의 계층에 대해 1,2,3,4는 각 타겟의 확률을 나타낸다. 이들의 최대 값은 확률의 confidence값으로 되어있다. 우리는 5개의 계층에 대해서 진행했고 각 계층별로 자세히 확인 가능하도록 그림을 그려줄 수 있다.<br />

In [ ]:
plt.rcParams['font.size'] = 18

# Kdeplot
g = sns.FacetGrid(predictions, row = 'fold', hue = 'Target', size = 3, aspect = 4)
g.map(sns.kdeplot, 'confidence');
g.add_legend();

plt.suptitle('Distribution of Confidence by Fold and Target', y = 1.05);

What we see here is that the confidence for each class if relatively low. It does appear that the model has greater confidence in Target=4 predictions which makes sense because of the class imbalance and the high prevalence of this label. <br />
=> 우리가 알수 있는 것은 각 클래스의 confidence가 상대적으로 낮다는 것이다. 우리가 이전에도 알아차렸던 것 처럼 이 문제에는 class imbalance문제가 있기 때문에 Target=4 에 대한 confidence가 굉장히 높다는 것을 알 수 있다.

Another way to look at the information is as a violinplot. This shows the same information, with the number of observations related to the width of the plot. <br />
=> 이를 보는 다른 방법은 바이올린 플롯을 이용하는 것이다.

In [ ]:
plt.figure(figsize = (24, 12))
sns.violinplot(x = 'Target', y = 'confidence', hue = 'fold', data = predictions);

Overall, these results show the issue with imbalanced class problems: our model cannot distinguish very well between the classes that are underrepresented. Later we'll look at predictions themselves and see where our model is "confused". For now, we can generate a submission file and submit it to the competition. <br />
=> 이 또한 우리가 직면한 클래스 불일치 문제가 있다. 우리의 모델은 빈도수가 적게 등장하는 클래스들을 잘 구분하지 못한다. 추후에 prediction을 보고 우리의 모델이 문제가 있다는것을 살펴보자. 일단은 제출을 해보도록하자.

When we actually make predictions for each household, we average the predictions from each of the folds. Therefore, we are essentially using multiple models since each one is trained on a slightly different fold of the data. The gradient boosting machine is already an ensemble machine learning model, and now we are using it almost as a meta-ensemble by averaging predictions from several gbms. <br />
=> 우리가 각 가구별로 예측치를 만들때, 우리는 각 폴드로부터 나오는 예측치들을 평균했다. 그러므로 각각의 모델이 데이터의 다른 계층으로 훈련되었기 때문에 여러개의 모델을 사용하는 것은 필수적이다. 그라디언트 부스팅 모델은 이미 앙상블 모델이고, 우리는 가리디언트 부스팅 모델을 다른 그라디언트 부스팅 모델들로부터 예측치들을 평균한 meta-ensemble로써 활용한다.

This process is shown in the code below.

In [ ]:
predictions.loc[predictions['idhogar'] == '000a08204',:]

In [ ]:
predictions.groupby('idhogar', as_index = False).mean().head()

In [ ]:
# Average the predictions over folds
predictions = predictions.groupby('idhogar', as_index = False).mean()

# Find the class and associated probability
predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
predictions = predictions.drop(columns = ['fold'])

In [ ]:
predictions.loc[predictions['idhogar'] == '000a08204',:]

In [ ]:
# Plot the confidence by each target
plt.figure(figsize = (10, 6))
sns.boxplot(x = 'Target', y = 'confidence', data = predictions);
plt.title('Confidence by Target');

plt.figure(figsize = (10, 6))
sns.violinplot(x = 'Target', y = 'confidence', data = predictions);
plt.title('Confidence by Target');

We can have the function instead return the actual submission file. This takes the average predictions across the five folds, in effectm combining 5 different models, each one trained on a slghtly different subset of the data. <br />
=> 우리는 실제 제출파일을 반환하는 함수를 가지고 있다. 이는 5개의 계층을 평균내었으며, 각각의 모델은 데이터의 다른 겹들로 훈련되어 약간씩 차이가 있다.

In [ ]:
%%capture
submission, gbm_fi, valid_scores = model_gbm(train_set, train_labels, 
                                             test_set, test_ids, return_preds=False)

submission.to_csv('gbm_baseline.csv')

In [ ]:
_ = plot_feature_importances(gbm_fi, threshold=0.95)

The gbm seems to think the most important features are those derived from ages. The education variables also show up in the most important features. <br />
=> 그라디언트 부스팅 모델은 나이를 가장 중요한 특징ㅇ라고 생각한 것같다. 교육 변수 또한 중요한 변수로 생각하고 있다.

## Try Selected Features
The next step with the LightGBM is to try the features that were selected through recursive feature elimination.<br />
다음으로는 우리가 Recursive Feature Elimination을 통해 얻은 피쳐들로 모델을 만들어 보자

In [ ]:
submission, gbm_fi_selected, valid_scores_selected = model_gbm(train_selected, train_labels, 
                                                               test_selected, test_ids)

In [ ]:
model_results

In [ ]:
model_results = model_results.append(pd.DataFrame({'model': ["GBM", "GBM_SEL"], 
                                                   'cv_mean': [valid_scores.mean(), valid_scores_selected.mean()],
                                                   'cv_std':  [valid_scores.std(), valid_scores_selected.std()]}),
                                                sort = True)
model_results.reset_index(drop=True)

In [ ]:
model_results_ = model_results.set_index('model')
model_results_['cv_mean'].plot.bar(color = 'orange', figsize = (8, 6),
                                  yerr = list(model_results['cv_std']),
                                 edgecolor = 'k', linewidth = 2)
plt.title('Model F1 Score Results');
plt.ylabel('Mean F1 Score (with error bar)');
# model_results.reset_index(inplace = True)

The massive advantage of the gradient boosting machine is on display here. For a final step, let's try using 10-folds with both sets and add them to the plot. <br />

In [ ]:
submission, gbm_fi, valid_scores = model_gbm(train_set, train_labels, test_set, test_ids, 
                                             nfolds=10, return_preds=False)

In [ ]:
submission.to_csv('gbm_10fold.csv', index = False)

In [ ]:
submission, gbm_fi_selected, valid_scores_selected = model_gbm(train_selected, train_labels, test_selected, test_ids,
                                                               nfolds=10)

In [ ]:
submission.to_csv('gmb_10fold_selected.csv', index = False)

In [ ]:
model_results

In [ ]:
model_results = model_results.append(pd.DataFrame({'model': ["GBM_10Fold", "GBM_10Fold_SEL"], 
                                                   'cv_mean': [valid_scores.mean(), valid_scores_selected.mean()],
                                                   'cv_std':  [valid_scores.std(), valid_scores_selected.std()]}),
                                    sort = True)

In [ ]:
model_results_ = model_results.set_index('model')
model_results_['cv_mean'].plot.bar(color = 'orange', figsize = (8, 6),
                                  yerr = list(model_results['cv_std']),
                                 edgecolor = 'k', linewidth = 2)
plt.title('Model F1 Score Results');
plt.ylabel('Mean F1 Score (with error bar)');

In [ ]:
model_results

The best model seems to be the Gradient Boosting Machine trained with 10 folds on the selected features. This model has not yet been optimized, but we might be able to get a little more performance through optimization. <br />
가장 최고의 모델은 선택된 특징들로 이용한 10겹 교차검증으로 훈련된 그라디언트 부스팅 모델인것 같습니다. [번역자는 그림에서 보다시피 다르지만 해석은 같게 하겠습니다.] 이 모델은 아직 최적화가 되어있지 않습니다. 그러나 최적화를 통해서 약간의 성능향상을 할 수 있을 것입니다.

In [ ]:
print(f"There are {gbm_fi_selected[gbm_fi_selected['importance'] == 0].shape[0]} features with no importance.")

Well that should make us feel better! All of the features we are using have some importance to the Gradient Boosting Machine. It might be a good idea to go back and retry feature selection but with the GBM since that is the model we are using. <br />
=> 그라디언트 부스팅 머신에 모든 피쳐들이 사용되고 있다는 것은 좋은 징조인 것 같습니다! 다시 돌아가서 feature selection을 재시도 하는 것은 좋은 생각인 것 같지만 우리가 사용하고 있는 GBM모델을 이용해야 할 것입니다.

The next step to take is Model Optimization, the process of getting the most from a machine learning model. <br />
다음 단계는 머신러닝모델에서 가장 중요한 모델 최적화 단계입니다

# Model Optimization
Model optimization is the process of extracting the best performance from a machine learning model by tuning the hyperparameters through cross-validation. This is necessary because the best model hyperparameters are different for every dataset. <br />
=> 모델 최적화는 머신러닝모델에서 하이퍼 파라미터를 교차검증을 통해 튜닝하여 최고의 성능을 뽑아내기 위한 과정입니다. 이는 최고의 모델 하이퍼파라미터가 데이터 세트들마다 다르기 때문에 필수적입니다.

In this notebook, we won't get too far into model tuning, but there are multiple options:
여기서는 모델 튜닝에 대해서 그렇게 깊게 다루지는 않겠지만 여러가지 옵션들이 있습니다.

1. Manual
2. Grid Search
3. Random Search
4. Automated Optimization

We will stick to 4. because it is generally the most efficient method and can easily be implemented in a number of libraries, including Hyperopt, which uses a modified version of Bayesian Optimization with the Tree Parzen Estimator.<br />
=> 우리는 네번째에 대해서 알아볼 것이며, 이는 가장 효율적인 방법이면서 많은 라이브러리들로 쉽게 실행가능하기 때문입니다. 우리는 Hyperopt를 사용할 것인데 이는 Tree Parzen Estimator를 활용한 Bayesian Optimization의 수정된 버전을 사용합니다.

## Model Tuning with Hyperopt
Bayesian optimization requires 4 parts: <br />
=> Bayesian optimization은 네가지 파트로 구성됩니다.

* Objective function: what we want to maximize (or minimize) <br />
최대화 최소화 시키려는 목적 함수 <br />
* Domain space: region over which to search <br />
찾으려는 region<br />
* Algorithm for choosing next hyperparameters: uses past results to suggest next values <br />
다음 파라미터를 고르는 알고리즘: 두번 째의 결과값을 값으로 사용함<br />
* Results history: saves the past results <br />
결과 history: 과거 결과들일 기록함 <br />

I've written previously about using Hyperopt, so here we'll stick to the implementation.<br />
우리는 이를 이용해서 실행해 봅시다 <br />


In [ ]:
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll.stochastic import sample

In [ ]:
import csv
import ast
from timeit import default_timer as timer

In [ ]:
def objective(hyperparameters,nfolds=5):
    
    global ITERATION
    ITERATION += 1
    
    subsample = hyperparameters['boosting_type'].get('subsample',1.0)
    subsample_freq = hyperparameters['boosting_type'].get('subsample_freq',0)
    
    boosting_type = hyperparameters['boosting_type']['boosting_type']
    
    if boosting_type == 'dart':
        hyperparameters['drop_rate'] = hyperparameters['boosting_type']['drop_rate']
        
    hyperparameters['subsample'] = subsample
    hyperparameters['subsample_freq'] = subsample_freq
    hyperparameters['boosting_type'] = boosting_type
    
    if not hyperparameters['limit_max_depth']:
        hyperparameters['max_depth'] = -1
        
    for parameter_name in ['max_depth','num_leaves','subsample_for_bin','min_child_samples','subsample_freq']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])
        
    if 'n_estimators' in hyperparameters:
        del hyperparameters['n_estimators']
    
    strkfold = StratifiedKFold(n_splits=nfolds, shuffle=True)
    
    features = np.array(train_selected)
    # ????
    labels = np.array(train_labels).reshape((-1))
    
    valid_scores = []
    best_estimators = []
    run_times =[]
    
    model = lgb.LGBMClassifier(**hyperparameters,class_weight='balanced',n_jobs=-1,metric='None',n_estimators=10000)
    
    for i,(train_indices,valid_indices) in enumerate(strkfold.split(features,labels)):
        
        X_train,X_valid = features[train_indices],features[valid_indices]
        y_train,y_valid = labels[train_indices],labels[valid_indices]
        
        start = timer()
        
        model.fit(X_train,y_train,early_stopping_rounds=100,eval_metric=macro_f1_score,eval_set=[(X_train,y_train),(X_valid,y_valid)],eval_names=['train','valid'],verbose=400)
        
        end = timer()
        
        valid_scores.append(model.best_score_['valid']['macro_f1'])
        best_estimators.append(model.best_iteration_)
        
        run_times.append(end-start)
        
    score = np.mean(valid_scores)
    score_std = np.std(valid_scores)
    loss = 1 - score
    
    run_time = np.mean(run_times)
    run_time_std = np.std(run_times)
    
    estimators = int(np.mean(best_estimators))
    hyperparameters['n_estimators'] = estimators
    
    of_connection = open(OUT_FILE,'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss,hyperparameters,ITERATION,run_time,score,score_std])
    of_connection.close()
    
    if ITERATION % PROGRESS == 0:
        display(f'Iteration: {ITERATION}, Current Score: {round(score, 4)}.')
    
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'time': run_time, 'time_std': run_time_std, 'status': STATUS_OK, 
            'score': score, 'score_std': score_std}

### 2. Search Space
The domain is the entire range of values over which we want to search. The only difficult part is the subsample ratio which must be set to 1.0 if the boosting_type="goss". Feel free to play around with the values here. <br />
=> 도메인은 우리가 찾고자하는 값들의 전체 범위입니다. 서브샘플 비율은 어려운 부분이며, 만약 boosting_type이 'goss'이면 무조건 1.0으로 세팅해야합니다. 자유롭게 도메인들을 둘러보도록 합시다.

> https://github.com/hyperopt/hyperopt/wiki/FMin [hyperopt에 대하여]

In [ ]:
# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                              [{'boosting_type': 'gbdt', 
                                'subsample': hp.uniform('gdbt_subsample', 0.5, 1),
                                'subsample_freq': hp.quniform('gbdt_subsample_freq', 1, 10, 1)}, 
                               {'boosting_type': 'dart', 
                                 'subsample': hp.uniform('dart_subsample', 0.5, 1),
                                 'subsample_freq': hp.quniform('dart_subsample_freq', 1, 10, 1),
                                 'drop_rate': hp.uniform('dart_drop_rate', 0.1, 0.5)},
                                {'boosting_type': 'goss',
                                 'subsample': 1.0,
                                 'subsample_freq': 0}]),
    'limit_max_depth': hp.choice('limit_max_depth', [True, False]),
    'max_depth': hp.quniform('max_depth', 1, 40, 1),
    'num_leaves': hp.quniform('num_leaves', 3, 50, 1),
    'learning_rate': hp.loguniform('learning_rate', 
                                   np.log(0.025), 
                                   np.log(0.25)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 2000, 100000, 2000),
    'min_child_samples': hp.quniform('min_child_samples', 5, 80, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.5, 1.0)
}

In [ ]:
sample(space)

### 3. Algorithm
The algorithm for choosing the next values is the Tree Parzen Estimator which uses Bayes rule for constructing a surrogate model of the objective function. Instead of maximizing the objective function, the algorithm maximizes the Expected Improvement (EI) of the surrogate model. <br />
=> 다음 값을 고르는 알고리즘은 목적함수의 임시 모델을 만들기위한 베이즈이론을 따르는 Tree Parzen Estimator이다. 목적함수를 최대화 시키는 것 대신에 알고리즘은 임시 모델의 Expected Improvement를 최대화시킨다. 


In [ ]:
algo = tpe.suggest

### 4. Results History
We'll use two different methods for recording results: <br />
결과를 기록하는데 있어서 두가지 방법을 사용할 것이다.

1. Trials object: stores everything returned from the objective function <br />
목적함수로부터 반환되는 모든 것을 저장 <br />
2. Write to a csv file on every iteration <br />
모든 반복들을 csv파일로 쓰기 <br />

I like using multiple methods for tracking progress because it means redundancy. One way may fail, but hopefully both will not! The csv file can be used to monitor the method while it is running and the Trials object can be saved and then reloaded to resume optimization. <br />
=> 나는 결과를 추적하는데 여러방법을 사용하는 것을 좋아하는데 왜냐하면 결과 추적은 중복의 수단이 되기 때문이다. 하나의 방법이 실패하면 다른 것은 실패하지 않을 것이다. csv파일은 실행되는 것을 관찰하는 방법이며, 이렇게 수행들의 객체는 저장되고 최적화를 재개하기 위해 다시 로드된다.

In [ ]:
trials = Trials()

OUT_FILE = 'optimization.csv'
of_connection = open(OUT_FILE,'w')
writer = csv.writer(of_connection)

MAX_EVALS = 100
PROGRESS = 10
N_FOLDS = 5
ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score', 'std']
writer.writerow(headers)
of_connection.close()

In [ ]:
display("Running Optimization for {} Trials.".format(MAX_EVALS))

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

To resume training, we can pass in the same trials object and increase the max number of iterations. For later use, the trials can be saved as json.

In [ ]:
import json

# Save the trial results
with open('trials.json', 'w') as f:
    f.write(json.dumps(str(trials)))

Using Optimized Model
Once the optimization procedure has finished, we can use the best results for modeling.

In [ ]:
results = pd.read_csv(OUT_FILE).sort_values('loss', ascending = True).reset_index()
results.head()

In [ ]:
plt.figure(figsize = (8, 6))
sns.regplot('iteration', 'score', data = results);
plt.title("Optimization Scores");
plt.xticks(list(range(1, results['iteration'].max() + 1, 3)));

In [ ]:
best_hyp = ast.literal_eval(results.loc[0, 'hyperparameters'])
best_hyp

In [ ]:
submission, gbm_fi, valid_scores = model_gbm(train_selected, train_labels, 
                                             test_selected, test_ids, 
                                             nfolds = 10, return_preds=False)

model_results = model_results.append(pd.DataFrame({'model': ["GBM_OPT_10Fold_SEL"], 
                                                   'cv_mean': [valid_scores.mean()],
                                                   'cv_std':  [valid_scores.std()]}),
                                    sort = True).sort_values('cv_mean', ascending = False)

In [ ]:
submission, gbm_fi, valid_scores = model_gbm(train_set, train_labels, 
                                             test_set, test_ids, 
                                             nfolds = 10, return_preds=False)

model_results = model_results.append(pd.DataFrame({'model': ["GBM_OPT_10Fold"], 
                                                   'cv_mean': [valid_scores.mean()],
                                                   'cv_std':  [valid_scores.std()]}),
                                    sort = True).sort_values('cv_mean', ascending = False)

In [ ]:
model_results.head()

In [ ]:
submission, gbm_fi, valid_scores = model_gbm(train_selected, train_labels, 
                                             test_selected, test_ids, 
                                             nfolds = 10, return_preds=False)

In [ ]:
submission.to_csv('gbm_opt_10fold_selected.csv', index = False)

At this point, to improve our performance, we can continue with the optimization, try more feature engineering, stack or ensemble additional models, or look at more experimental methods such as dimension reduction or oversampling. However, the scores we are getting are fairly good and I'm going to move on to a crucial part of machine learning: investigating the predictions to see where our model is wrong. <br />
우리의 성능을 높이기 위해 최적화를 계속할 수 있고, 다른 feature engineering을 할수도 있고 추가적인 스태킹 또는 앙상블 모델이나 또는 차원축소 또는 오버샘플링과 같은 실험적인 방법을 할수 있습니다. 그러나 점수는 생각보다도 좋으며, 그래서 머신러닝에 있어서 좀더 중요한 부분인, 예측치에서 우리의 모델이 잘못된 부분을 조사해보려고 한다.<br />

While Kaggle is great for presenting realistic datasets, the methods needed to get to the very top of the leaderboard are not generally used in the real-world. Past some level of performance, accuracy takes a back seat to interpretability. People want to know why a model makes the predictions it does, much as they would want a human to be able to explain her decisions. We could work on squeezing some more performance from our model, but right now, our energy is better spent investigating our model. <br />
캐글은 실제 데이터셋을 제공하는 좋은 곳이지만, 실제로 높은 점수를 차지하는 사람들이 하는 방법들은 실제로 사용되지 않는다. 일정 수준의 성능에서, 정확도는 의미를해석하는 것보다 뒷전에 있다. 사람들은 왜 모델이 예측을 그렇게 했는지 알고 싶어한다. 우리는 모델의 더좋은 성능을 위해 짜낼 수 있지만, 그것 보다는 우리의 에너지를 우리의 모델을 조사하는데 초점을 맞춰보도록 하자.

In [ ]:
_ = plot_feature_importances(gbm_fi)

## Investigate Predictions
As a first attempt at looking into our model, we can visualize the distribution of predicted labels on the test data. We would expect these to show the same distribution as on the training data. Since we are concerned with household predictions, we'll look at only the predictions for each house and compare with that in the training data. <br />
=>우리의 모델을 보는 첫번째 시도로, 우리는 테스트데이터에 대한 예측된 라벨값들의 분포를 시각화할 수 있습니다. 우리는 이 분포가 훈련세트의 라벨분포와 동일하기를 바라고 있습니다. 우리가 가구예측을 해왔던것처럼, 우리는 각각의 가구의 데이터를 훈련세트의 라벨과 비교할 것입니다.

The following histrograms are normalize meaning that they show the relative frequency instead of the absolute counts. This is necessary because the raw counts differ in the training and testing data. <br />
=> 다음의 히스토그램은 절대적인 카운트가 아닌 상대적인 빈도를 나타내고 있습니다. 왜냐하면 실제적인 카운트는 테스트셋과 훈련셋이 다르기 때문입니다.

In [ ]:
preds = submission_base.merge(submission,on='Id',how='left')
preds = pd.DataFrame(preds.groupby('idhogar')['Target'].mean())

fig,axes = plt.subplots(1,2,sharey=True,figsize=[12,6])
heads['Target'].sort_index().plot.hist(normed=True,edgecolor=r'k',linewidth=2,ax=axes[0])
axes[0].set_xticks([1,2,3,4])
axes[0].set_xticklabels(poverty_mapping.values(),rotation=60)
axes[0].set_title('Train Label Distribution')


preds['Target'].sort_index().plot.hist(normed = True, 
                                       edgecolor = 'k',
                                       linewidth = 2,
                                       ax = axes[1])
axes[1].set_xticks([1, 2, 3, 4]);
axes[1].set_xticklabels(poverty_mapping.values(), rotation = 60)
plt.subplots_adjust()
plt.title('Predicted Label Distribution');

In [ ]:
heads['Target'].value_counts()

In [ ]:
preds['Target'].value_counts()

The predicted distribution looks close to the training distribution although there are some differences. Depending on the run of the notebook, the results you see may change, but for this edition, the 4s are underrepresented in the predictions and the 3s are overrepresented.

One potentially method for dealing with imbalanced classification problems is oversampling the minority class, which is easy to do in Python using the imbalanced learn library. We won't explore that option here.